<a href="https://colab.research.google.com/github/hatopopvr/MyBSAnalytics_Lite/blob/main/MyBSAnalytics_Lite_En.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MyBSAnalytics_Lite_En
<b>Data</b>
- Score Data from ScoreSaber Public API - [doc](https://docs.scoresaber.com/)  
- Ranked Map Data from RankedMapData by rakkyo150 - [RankedMapData](https://github.com/rakkyo150/RankedMapData)  
- Various ScoreSaber Rankings Data by rynan4818 - [ScoreSaberRanking
](https://github.com/rynan4818/ScoreSaberRanking)
- Cover Image from - https://cdn.scoresaber.com/covers/{hash}.png  

<b>Author</b>
- hatopop ([@hatopop_vr](https://twitter.com/hatopop_vr))

<b>Caution</b>
- If you would like me to add a default value for the timezone, please mentions me on Twitter.

In [ ]:
#@title # Input Information
#@markdown ---
#@markdown <h4>Input Data</h4>

#@markdown <font size="3">`player_id` : PlayerID for ScoreSaber. <b>Must</b> be changed.<br/></font>
player_id =  76561198412839195#@param {type:"number"}

#markdown <font size="3">`google_drive_mount_path` ：GoogleDriveのマウント先です。変更不要。</font><br/>
google_drive_mount_path = "/content/drive" #param {type:"string"}

#@markdown <font size="3">`google_drive_dir_path` : The directory to save data in GoogleDrive. Optional change.</font>
google_drive_dir_path = "/MyDrive/MyBeatSaberAnalysis/data" #@param {type:"string"}

#@markdown ---
#@markdown <h4>Setting</h4>

#@markdown <font size="3">`saved_player_score_is_enable`: Whether to use saved score data. Download only the difference.</font>
saved_player_score_is_enable = True #@param {type:"boolean"}
#@markdown <font size="3">`acc_recalq_override_is_enable`: Whether to use the recalculated Accuracy value based on the number of Notes and Combo.</font>
acc_recalq_override_is_enable = True #@param {type:"boolean"}
#markdown <font size="3">`ranked_song_form_leaderboard_is_enable`: Ranked譜面クリア進捗用のレベル別譜面数をScoreSaberのLeaderBoardから取得し直すか。☑で使用。<br /></font>
ranked_song_from_leaderboard_is_enable = False
#markdown <font size="3">`ss_plus_is_enable`: AccRank区分にSS+を使用するか。☑で使用。  </font><br />
ss_plus_is_enable = True #param {type:"boolean"}
#markdown <font size="3">`ss_plus_val`:SS+の設定値です。必要に応じて変更ください。範囲は91-99。
ss_plus_val = 95 #param {type:"slider", min:91, max:99, step:1}
#@markdown <font size="3">`latest`:The number of days to be treated as the most recent results. 0 is the same date as today in timezone. If 1, from yesterday.
latest =  1#@param {type:"integer"}

#@markdown <font size="3">`timezone`:Input or Select your timezone.<br />
#@markdown <font size="3"> The following URL can be used as a reference for time zones ([mjrulesamrat/countryinfo.py](https://gist.github.com/mjrulesamrat/0c1f7de951d3c508fb3a20b4b0b33a98))
timezone = "US/Pacific" #@param ["Asia/Tokyo", "US/Pacific", ""] {allow-input: true}
#@markdown <font size="3">`score_mode`:Select the score you want to get, where `HigherScore` is the higher of `BaseScore` and `ModifieredScore`.</font>
score_mode = "HigherScore" #@param ["BaseScore", "ModifieredScore", "HigherScore"]

#@title Libraryの取得
!pip install GitPython
!pip install numpy==1.23.5 #numpyメジャーアプデ適応による不具合修正のためnumpyのダウングレード 2.0.2 -> 1.25.0
!pip install pandas==1.5.0  # ダウングレード、日付のエラーとappendのエラー対応
import warnings
warnings.filterwarnings("ignore")
import os
import shutil
import pandas as pd
from pandas import json_normalize
import json
from datetime import datetime, timedelta
from dateutil import tz
import plotly.express as px
from ipywidgets import interact, Select, BoundedIntText, IntSlider, ToggleButtons, Layout, HBox, VBox, AppLayout
import time
from tqdm import tqdm
import requests
import math
import git
import gc
from IPython.display import HTML, Javascript,Image
from google.colab import drive, files
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import copy

#@title Timezone UTC->日本時間+9:00での実行日時の取得(tz_ja)
# tz_ja = pd.Timestamp(datetime.now()).tz_localize('UTC').tz_convert('Asia/Tokyo')
tz_ja = pd.Timestamp(datetime.now()).tz_localize('UTC').tz_convert(timezone)
#today_tz_ja = datetime_now_tz_ja.strftime("%Y.%m.%d")
# print("get_date:{} timezone:{}".format(tz_ja, timezone))
#@title Google Driveのマウント
drive.mount(google_drive_mount_path)

#@title その他内部設定値
# データ元のURL
## song_data_zip_git_url: ScoreSaberの全曲情報のzip(json)のURLです。変更なければそのままで。
song_data_zip_git_url = "https://github.com/andruzzzhka/BeatSaberScrappedData.git"
## ranked_excluded_data_git_url: ScoreSaberのRank譜面除外リスト(csv)のURLです。暫定的処置。
ranked_excluded_data_git_url = "https://github.com/hatopopvr/ScoreSaberRankedExcludedMaps.git"
## rankedmapdata_url: BeatSaverデータのcsvのURLです。らっきょさんデータ。
rankedmapdata_url = 'https://api.github.com/repos/rakkyo150/RankedMapData/releases'
# google drive内のdata置き場親フォルダ
data_path = r"{}{}".format(google_drive_mount_path, google_drive_dir_path)
# player情報の親フォルダ(data_pathの子フォルダ)
player_path = r"{}/players_data/{}".format(data_path, player_id)
## playerinfoの保存先
player_info_path = r"{}/player_info_{}.csv".format(player_path, player_id)
## playerのscore関連保存先
player_score_path = r"{}/scores_full_{}.csv".format(player_path, player_id)
player_ranked_path = r"{}/scores_ranked_{}.csv".format(player_path, player_id)
## playerのscore関連保存先
player_score_pickle_path = r"{}/scores_full_{}.pkl".format(player_path, player_id)
player_ranked_pickle_path = r"{}/scores_ranked_{}.pkl".format(player_path, player_id)
## 曲情報の保存関連
song_clone_path = r"/content/BeatSaberScrappedData"
song_zip_path = r"{}/combinedScrappedData.zip".format(song_clone_path)
song_json_path = r"{}/combinedScrappedData.json".format(data_path)
## 曲情報の保存先
song_list_path = r"{}/song_list_full.csv".format(data_path)
song_ranked_path = r"{}/song_ranked.csv".format(data_path)
# ランク除外関連パス
ranked_excluded_clone_dir_path = r"/content/ScoreSaberRankedExcludedMaps"
ranked_excluded_clone_csv_path = r"{}/RankedExcludedMaps.csv".format(ranked_excluded_clone_dir_path)
ranked_excluded_csv_path = r"{}/RankedExcludedMaps.csv".format(data_path)
# levelclearランク除外関連パス
level_cleared_path = r"{}/level_cleared_{}.csv".format(player_path, player_id)

# playlistの保存
song_playlist_path = r"{}/playlists".format(data_path)
song_worst_playlist_path = r"{}/worst_playlist_{}.json".format(song_playlist_path, tz_ja.strftime("%Y%m%d"))
song_top_playlist_path = r"{}/top_playlist_{}.json".format(song_playlist_path, tz_ja.strftime("%Y%m%d"))

# SS_plus設定値
ss_plus = "SS+{}".format(ss_plus_val)
ss_plus_rate = "SS+{}-Rate".format(ss_plus_val)
# その他 colab表示など設定
pd.options.display.precision = 2
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
page_count=100 #ScoreSaberの1ページあたりのページ数

# 出力最大高さ
def resize_colab_cell():
    display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))

get_ipython().events.register('pre_run_cell', resize_colab_cell)

#@title 列情報の設定
# Player Infoの記録用列 (TotalFC, RankedFCは別途結合)
cols_info =[
    "Pic"
    ,"name"
    ,"country"
    ,"pp"
    ,"rank"
    ,"countryRank"
    ,"role"
    ,"TotalScore"
    ,"RankedScore"
    ,"AveRankedAcc"
    ,"TotalPlay"
    ,"RankedPlay"
    ,"ReplayWatched"
    ,"ScoreDate"
    ,"TotalFC"
    ,"RankedFC"
    ,"TotalPlayRank"
    ,"TotalPlayJPRank"
    ,"RankedPlayRank"
    ,"RankedPlayJPRank"
    ,"TotalScoreRank"
    ,"TotalScoreJPRank"
    ,"RankedScoreRank"
    ,"RankedScoreJPRank"
    ,"AveRankedAccRank"
    ,"AveRankedAccJPRank"
]

# Player Infoの表示用列 (TotalFC, RankedFCは別途結合)
cols_info_sort =[
    "Pic"
    ,"name"
    ,"country"
    ,"pp"
    ,"rank"
    ,"countryRank"
    ,"role"
    ,"TotalScore"
    ,"RankedScore"
    ,"AveRankedAcc"
    ,"TotalPlay"
    ,"RankedPlay"
    ,"TotalFC"
    ,"RankedFC"
    ,"ReplayWatched"
    ,"ScoreDateTz"
]

# リュナンさんのScoreSaberRanking表示用列(不使用)
cols_info_rank = [
    "TotalPlayRank"
    ,"TotalPlayJPRank"
    ,"RankedPlayRank"
    ,"RankedPlayJPRank"
    ,"TotalScoreRank"
    ,"TotalScoreJPRank"
    ,"RankedScoreRank"
    ,"RankedScoreJPRank"
    ,"AveRankedAccRank"
    ,"AveRankedAccJPRank"
]

# summary用列
cols_summary =[
    "rank"
    ,"countryRank"
    ,"pp"
    ,"AveRankedAcc"
    ,"TotalPlay"
    ,"RankedPlay"
]

# SongList抜粋用列
cols_song =[
    "Cover"
    ,"Song"
    ,"Level"
    ,"LevelStr"
    ,"Stars"
    # ,"maxPP"
    # ,"maxScore"
    ,"Difficulty"
    # ,"Play"
    # ,"DailyPlay"
    ,"CreatedDateJa"
    ,"RankDateJa"
    ,"Bombs"
    ,"Notes"
    ,"Obstacles"
    ,"Njs"
    ,"NjsOffset"
    ,"Bpm"
    ,"Upvotes"
    #,"Downvotes"
    ,"Duration"
    ,"Ranked"
]

# Rank譜面除外リスト結合用列
cols_excluded = [
                 'Hash',
                 'Difficulty',
                 'RankedExcluded'
                 ]

# Score用列
cols_score =[
    "Cover"
    ,"Song"
    ,"Level"
    ,"Stars"
    ,"Acc"
    ,"AccRank"
    ,"FC"
    ,"Rank"
    ,"PP"
    ,"Miss"
    ,"Bad"
    ,"Combo"
    ,"Score"
    #,"Mode"
    ,"Difficulty"
    ,"Play"
    ,"DailyPlay"
    ,"Bpm"
    ,"Duration"
    ,"Notes"
    ,'Nps'
    ,"Njs"
    ,"Bombs"
    ,"Obstacles"
    # ,"NjsOffset"
    ,'Upvotesratio'
    ,"Upvotes"
    ,"Downvotes"
    ,"Ranked"
    ,"Days"
    ,"Months"
    ,"Tags"
    ,"Preview"
 ]

# Acc再計算用列
cols_recalq = [
    'Hash',
    'Cover',
    'SongName',
    'Difficulty', 'Stars',
    # 'Ranked',
    'Notes',
    'Acc',
    'AccRecalq',
    'AccDiff',
    'AccRank',
    'AccRankRecalq',
    'MaxScore',
    'MaxScoreRecalq',
    'MaxScoreDiff',
    'Score',
    'Miss',
    'Combo',
    # 'RankDateJa',
    # 'CreatedDateJa',
    'Preview'
]

# ScatterPlot用X-Y軸列
cols_xy = [
    "Stars"
    ,"Level"
    ,"Acc"
    ,"Rank"
    ,"PP"
    ,"Miss"
    ,"Bad"
    ,"Combo"
    ,"Score"
    ,"Play"
    ,"DailyPlay"
    ,"Days"
    ,"Months"
    ,"Bpm"
    ,"Duration"
    ,"Notes"
    ,'Nps'
    ,"Njs"
    ,"Bombs"
    ,"Obstacles"
    ,"NjsOffset"
    ,'Upvotesratio'
    ,"Upvotes"
    ,"Downvotes"
]

# RankedMap(BeatSaver)結合用列
cols_rankedmap = ['Hash',
                  'Difficulty',
                  'Upvotesratio',
                  'Nps',
                  'Tags', 'Preview'
]

# ScatterPlot用カテゴリ
cols_cate = [
    "LevelStr"
    ,"Days"
    ,"DaysStr"
    ,"Months"
    ,"MonthsStr"
    ,"LatestStr"
    ,"Acc"
    ,"AccRank"
    ,"FC"
    ,"SongAuthor"
    ,"LevelAuthor"
]

# Worst-Top ScatterPlot用カテゴリ
cols_cate_wt = [
    "WorstTop"
    ,"LevelStr"
    ,"Days"
    ,"DaysStr"
    ,"Months"
    ,"MonthsStr"
    ,"LatestStr"
    ,"Acc"
    ,"AccRank"
    ,"FC"
    ,"SongAuthor"
    ,"LevelAuthor"
]

# Latest_history用の列
cols_latest_history = ['Cover', 'Song', 'Difficulty', 'Stars', 'Acc', 'AccRank', 'FC', 'Rank', 'PP',
       'Miss', 'Bad', 'Combo', 'Score']


cols_worst_n = ['Cover', 'Song', 'Difficulty', 'Level', 'Stars', 'Acc', 'AccRank', 'FC', 'Rank', 'PP',
       'Miss', 'Bad', 'Combo', 'Score']

#@title スタイルの設定

cover_image_size=70 #カバー,プロファイル画像のサイズ単位px
info_image_size=150 #Player画像のサイズ単位px

style_format = {
        "rank": "#{:,.0f}",
        "countryRank": "#{:,.0f}",
        "TotalScore": "{:,.0f}",
        "RankedScore": "{:,.0f}",
        "AveRankedAcc": "{:,.2f}%",
        "TotalPlay": "{:,.0f}",
        "TotalFC": "{:,.0f}",
        "RankedFC": "{:,.0f}",
        "ReplayWatched": "{:,.0f}",
        "Level": "{:,.0f}",
        "Stars": "{:,.2f}★",
        "Acc": "{:,.2f}%",
        "AccRecalq": "{:,.2f}%",
        "AccDiff": "{:,.2f}%",
        "MaxScore": "{:,.0f}",
        "MaxScoreRecalq": "{:,.0f}",
        "MaxScoreDiff": "{:,.0f}",
        "Score": "{:,.0f}",
        "Rank": "#{:,.0f}",
        "pp": "{:,.2f}pp",
        "PP": "{:,.2f}pp",
        "Miss": "{:,.0f}",
        "Bad": "{:,.0f}",
        "Combo": "{:,.0f}",
        "Score": "{:,.0f}",
        "Play": "{:,.0f}",
        "DailyPlay": "{:,.0f}",
        "Bpm": "{:,.0f}",
        "Duration": "{:,.0f}",
        "Notes": "{:,.0f}",
        "Nps": "{:,.2f}",
        "Njs": "{:,.0f}",
        "Bombs": "{:,.0f}",
        "Obstacles": "{:,.0f}",
        "NjsOffset": "{:,.2f}",
        "Upvotesratio": "{:,.2f}",
        "Upvotes": "{:,.0f}",
        "Downvotes": "{:,.0f}",
        "Days": "{:,.0f}",
        "Months": "{:,.0f}",
        "Song": "{:,.0f}",
        "RecentCleared": "{:,.0f}",
        "Cleared": "{:,.0f}",
        "NF": "{:,.0f}",
        "NotCleared": "{:,.0f}",
        "AlreadyCleared": "{:,.0f}",
        "NotClearedRate": "{:,.2f}%",
        "AlreadyClearedRate": "{:,.2f}%",
        "RecentClearedRate": "{:,.2f}%",
        "NFRate": "{:,.2f}%",
        "SSS": "{:,.0f}",
        ss_plus: "{:,.0f}",
        "SS": "{:,.0f}",
        "S": "{:,.0f}",
        "A": "{:,.0f}",
        "B": "{:,.0f}",
        "Other": "{:,.0f}",
        "SSS-Rate": "{:,.2f}%",
        "SS-Rate": "{:,.2f}%",
        ss_plus_rate: "{:,.2f}%",
        "S-Rate": "{:,.2f}%",
        "A-Rate": "{:,.2f}%",
        "B-Rate": "{:,.2f}%",
        "Other-Rate": "{:,.2f}%",
        "FC": "{:,.0f}",
        "RecentFC": "{:,.0f}",
        "AlreadyFC": "{:,.0f}",
        "NotFC": "{:,.0f}",
        "AlreadyFCRate": "{:,.2f}%",
        "RecentFCRate": "{:,.2f}%",
        "NotFCRate": "{:,.2f}%",
        "Song": "{:,.0f}",
        "TotalPlayRank": "#{:,.0f}",
        "TotalPlayJPRank": "#{:,.0f}",
        "RankedPlayRank": "#{:,.0f}",
        "RankedPlayJPRank": "#{:,.0f}",
        "TotalScoreRank": "#{:,.0f}",
        "TotalScoreJPRank": "#{:,.0f}",
        "RankedScoreRank": "#{:,.0f}",
        "RankedScoreJPRank": "#{:,.0f}",
        "AveRankedAccRank": "#{:,.0f}",
        "AveRankedAccJPRank": "#{:,.0f}",
     }

ss_plus = "SS+{}".format(ss_plus_val)
ss_plus_rate = "SS+{}-Rate".format(ss_plus_val)

style_worst_top = {
        "Level": "{:,.0f}",
        "Stars": "{:,.2f}★",
        "Acc": "{:,.2f}%",
        "Rank": "#{:,.0f}",
        "pp": "{:,.2f}pp",
        "PP": "{:,.2f}pp",
        "Miss": "{:,.0f}",
        "Bad": "{:,.0f}",
        "Combo": "{:,.0f}",
        "Score": "{:,.0f}",
        "Play": "{:,.0f}",
        "DailyPlay": "{:,.0f}",
        "Bpm": "{:,.0f}",
        "Duration": "{:,.0f}",
        "Notes": "{:,.0f}",
        "Nps": "{:,.2f}",
        "Njs": "{:,.0f}",
        "Bombs": "{:,.0f}",
        "Obstacles": "{:,.0f}",
        "NjsOffset": "{:,.2f}",
        "Upvotesratio": "{:,.2f}",
        "Upvotes": "{:,.0f}",
        "Downvotes": "{:,.0f}",
        "Days": "{:,.0f}",
        "Months": "{:,.0f}",
     }

def color_negative_red(val):
    color = 'red' if val < 0 else 'blue'
    return 'color: %s' % color

# Difficulty color
def color_difficulty(val):
    if val == "Easy":
        return 'color: #66BB6A; font-weight: bold;'
        color = "#66BB6A"
    elif val == "Normal":
        return 'color: #29B6F6; font-weight: bold;'
        color = "#29B6F6"
    elif val == "Hard":
        return 'color: #FB8C00; font-weight: bold;'
        color = "#FB8C00"
    elif val == "Expert":
        return 'color: #E53935; font-weight: bold;'
        color = "#E53935"
    elif val == "Expert+":
        return 'color: #8E24AA; font-weight: bold;'
        color = "#8E24AA"
    elif val == "ExpertPlus":
        return 'color: #8E24AA; font-weight: bold;'
        color = "#8E24AA"
    return 'color: %s' % color

# Acc Rank color
def color_acc_rank(val):
    if val == "SSS":
        return 'color: #00ffff; font-weight: bold;'
    elif ss_plus_is_enable and val == ss_plus:
        return 'color: #636EFA; font-weight: bold;'
    elif val == "SS":
        return 'color: #ff4500; font-weight: bold;'
    elif val == "S":
        return 'color: #ffaaff; font-weight: bold;'
    elif val == "A":
        return 'color: #ffcc66; font-weight: bold;'
        #color = "#eebb55"
    elif val == "B":
        return 'color: #cccc66; font-weight: bold;'
    else:
        return 'color: #666666; font-weight: bold;'
    # return 'color: %s' % color

# FC color
def color_fc(val):
    if val == "FC":
        color = "#23D160"
    else:
        color = "black"
    return 'color: %s' % color

def color_fc(val):
    if val == "FC":
        return 'color: #23D160; font-weight: bold;'
    else:
        return 'color: black'


# Player Info Table Format
styles_info = [
    dict(selector="td", props=[("font-size", "130%"),
                               ("text-align", "center"),
                               ("padding-top", "0px"),
                               ("padding-bottom", "0px"),
                               ])
]

# Player Info Table Format
styles_data = [
    {"selector":"td", "props":[("padding-top", "0px"),
                               ("padding-bottom", "0px"),
                               ]},
    {"selector":"th", "props":[("padding-top", "0px"),
                               ("padding-bottom", "0px"),
                               ]},
]

# タイムゾーンの設定-------------------------


#@title 各種フォルダの作成
# 各種パスのディレクトリ作成
## データ大元のフォルダ作成
if os.path.exists(data_path) == False:
    print('MyBeatSaberAnalytics用のデータ格納フォルダをGoogle Driveに新規作成します。')
    print('データ格納フォルダ:{}'.format(data_path))
    os.makedirs(data_path, exist_ok=True)
    print('作成が完了しました。')

## データ大元のフォルダ作成
if os.path.exists(player_path) == False:
    old_player_info_path = r"{}/player_info_{}.csv".format(data_path, player_id)
    print('PlayerID:{}用のデータ格納フォルダを新規作成します。'.format(player_id))
    print('playerフォルダ:{}'.format(player_path))
    print('作成が完了しました。')
    os.makedirs(player_path, exist_ok=True)
    if os.path.isfile(old_player_info_path):
        print('データ引継ぎのため、player_infoｂのcsvファイルを上記フォルダに移動させます。')
        print('移動前:{}'.format(old_player_info_path))
        print('移動後:{}'.format(player_info_path))
        shutil.move(old_player_info_path, player_info_path)
        print('移動が完了しました。')

## プレイヤー用のフォルダ作成
if os.path.exists(song_playlist_path) == False:
    print('Playlist格納用のフォルダを新規作成します。')
    print('Playlist格納フォルダ:{}'.format(song_playlist_path))
    os.makedirs(song_playlist_path, exist_ok=True)
    print('作成が完了しました。')

#@title Get Player Info (df_info)
url = r"https://scoresaber.com/api/player/{}/full".format(player_id)
response = requests.get(url)
res_data = response.json()
df_info = json_normalize(res_data)

#df_info["Pic"] = '<img src="'+df_info["profilePicture"]+'"/>'
df_info["Pic"] = '<img src="'+df_info["profilePicture"]+'" style="width:{}px;"/>'.format(info_image_size)
df_info["TotalScore"] = df_info["scoreStats.totalScore"]
df_info["RankedScore"] = df_info["scoreStats.totalRankedScore"]
df_info["AveRankedAcc"] = df_info["scoreStats.averageRankedAccuracy"]
df_info["TotalPlay"] = df_info["scoreStats.totalPlayCount"]
df_info["RankedPlay"] = df_info["scoreStats.rankedPlayCount"]
df_info["ReplayWatched"] = df_info["scoreStats.replaysWatched"]
df_info["ScoreDate"] = datetime.now().strftime("%Y/%m/%d %H:%M:%S")

df_info["ScoreDateUtc"] = pd.to_datetime(df_info['ScoreDate'], utc=True)
_df_info_idx = df_info.set_index("ScoreDateUtc")
df_info["ScoreDateTz"] = _df_info_idx.index.tz_convert(timezone)

PlayCount = df_info["TotalPlay"][0]
RankedPlay = df_info["RankedPlay"][0]
RangeCount = math.ceil(PlayCount / page_count) + 1

print("------------------------------------")
print("timezone:{} | datetime:{} ".format(timezone, tz_ja))
print("Player:{}, TotalPlayCount:{:,}, RankedPlayCount:{:,}, PageCount:{}".format(df_info["name"][0], PlayCount, RankedPlay, RangeCount))

display(df_info[['Pic','name','TotalPlay']].style.set_table_styles(styles_info).format(style_format, na_rep="-"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
------------------------------------
timezone:US/Pacific | datetime:2025-06-15 01:58:06.786000-07:00 
Player:hatopop, TotalPlayCount:5,304, RankedPlayCount:4,410, PageCount:55


,Pic,name,TotalPlay
0,,hatopop,"5,304"


# Get Data

In [ ]:
#@title Get Ranked Song Data
#@markdown ScoreSaber's Ranked Song Data from [RankedMapData](https://github.com/rakkyo150/RankedMapData) by [rakkyo150](https://twitter.com/rakkyo150)

headers = {
    'Accept': 'application/vnd.github.v3+json',
}

response = requests.get(rankedmapdata_url, headers=headers)

data = response.json()

# 最新releaseのcsvのurl取得
url_rankmap_data = data[0]["assets"][0]["browser_download_url"]

file_name = os.path.join(data_path, os.path.basename(url_rankmap_data))
result = requests.get(url_rankmap_data, stream=True)
if result.status_code == 200:
    with open(file_name, 'wb') as file:
        result.raw.decode_content = True
        shutil.copyfileobj(result.raw, file)

df_rankmap_data = pd.read_csv(file_name)
df_rankmap_data = df_rankmap_data[[x for x in df_rankmap_data.columns if not x.startswith("Unnamed")]]
df_rankmap_data["hash"] = df_rankmap_data["hash"].str.upper()
df_rankmap_data["Preview"] = '<audio src="' + df_rankmap_data["previewUrl"] + '" preload="none" controls></audio>'

df_rankmap_data = df_rankmap_data.rename(columns=lambda x: x.capitalize())

df_rankmap_data.rename(columns={
                        "Songname":"SongName",
                        "Songsubname":"SongSub",
                        "Songauthorname":"SongAuthor",
                        "Levelauthorname":"LevelAuthor"
                            }, inplace=True)


# 集計異常対応 (bsr 1, me & uのSongAuthorがNaNの値の対応)
df_rankmap_data['SongAuthor'].fillna('Unknown', inplace=True)

# 準備
df_rankmap_data["Song"] = df_rankmap_data["SongName"] + " / " + df_rankmap_data["SongAuthor"] + " [" + df_rankmap_data["LevelAuthor"] + "]"
df_rankmap_data["Level"] = df_rankmap_data["Stars"].astype("int")
df_rankmap_data["LevelStr"] = df_rankmap_data["Level"].astype("str")

print("path:{},count:{:,}".format(file_name,len(df_rankmap_data["Hash"])))

<IPython.core.display.Javascript object>

path:/content/drive/MyDrive/MyBeatSaberAnalysis/data/outcome.csv,count:5,291


In [ ]:
#@title Get Ranked Song Data from Leaderboard
err_scoresaber_ranked_page = 350 #@param {type:"number"}
err_scoresaber_ranked_songs_min = 3000 #@param {type:"number"}
err_scoresaber_ranked_songs_max = 10000 #@param {type:"number"}

url = r"https://scoresaber.com/api/leaderboards?ranked=true&page=1&withMetadata=true"
response = requests.get(url)
total_count_from_leaderboard = response.json()['metadata']['total']
level_count_page = response.json()['metadata']['itemsPerPage']
scoresaber_ranked_page_count = math.ceil(total_count_from_leaderboard / level_count_page)
print("LeaderBoard Ranked Song:{:,}, Ranked Page:{:,}".format(total_count_from_leaderboard, scoresaber_ranked_page_count))

if len(df_rankmap_data["Hash"]) != total_count_from_leaderboard:
    print('ranked songs count is unmached.')
    res_data = response.json()
    df_ranked_songs_from_leaderboard = json_normalize(res_data['leaderboards'])

    # 例外時にページ数を修正する処理
    if (total_count_from_leaderboard < err_scoresaber_ranked_songs_min) or (total_count_from_leaderboard > err_scoresaber_ranked_songs_max):
        scoresaber_ranked_page_count = err_scoresaber_ranked_page
        print(f'leaderboardのmetadataのランク譜面数が異常です.ページ数を{scoresaber_ranked_page_count}として取得します')

    for i in tqdm(range(2, scoresaber_ranked_page_count+1)):
        url = r"https://scoresaber.com/api/leaderboards?ranked=true&page={}".format(i)
        try:
            response = requests.get(url)
            res_data = response.json()
            df_ranked_songs_from_leaderboard=df_ranked_songs_from_leaderboard.append(json_normalize(res_data['leaderboards']), ignore_index=True)
            # 例外時に抜ける処理
            if len(res_data['leaderboards']) == 0:
                print('break', i, len(res_data['leaderboards']), len(df_ranked_songs_from_leaderboard))
                break

        except:
            break

    def func_mode(x):
        if  x == "SoloStandard":
            return "Standard"
        else:
            return x

    df_ranked_songs_from_leaderboard['Hash'] = df_ranked_songs_from_leaderboard['songHash'].str.upper()
    df_ranked_songs_from_leaderboard['Song'] = df_ranked_songs_from_leaderboard['songName'] + " " + df_ranked_songs_from_leaderboard['songSubName'] + " / " + df_ranked_songs_from_leaderboard['songAuthorName'] + " [" + df_ranked_songs_from_leaderboard['levelAuthorName'] + "]"
    df_ranked_songs_from_leaderboard['SongName'] = df_ranked_songs_from_leaderboard['songName']
    df_ranked_songs_from_leaderboard['SongSub'] = df_ranked_songs_from_leaderboard['songSubName']
    df_ranked_songs_from_leaderboard['SongAuthor'] = df_ranked_songs_from_leaderboard['songAuthorName']
    df_ranked_songs_from_leaderboard['LevelAuthor'] = df_ranked_songs_from_leaderboard['levelAuthorName']

    df_ranked_songs_from_leaderboard['Mode'] = df_ranked_songs_from_leaderboard['difficulty.gameMode'].apply(func_mode)
    #df_scores['Mode'] = df_scores['leaderboard.difficulty.gameMode']
    _df_ranked_songs_from_leaderboard = df_ranked_songs_from_leaderboard['difficulty.difficultyRaw'].str.split('_', expand=True)
    _df_ranked_songs_from_leaderboard.columns = ['_','Difficulty', 'Mode']
    df_ranked_songs_from_leaderboard['Difficulty'] = _df_ranked_songs_from_leaderboard['Difficulty']
    df_ranked_songs_from_leaderboard['Stars'] = df_ranked_songs_from_leaderboard['stars']
    df_ranked_songs_from_leaderboard['Level'] = df_ranked_songs_from_leaderboard['Stars'].astype('int')
    df_ranked_songs_from_leaderboard["LevelStr"] = df_ranked_songs_from_leaderboard['Level'].astype('str')
    print("RankedSong(ScoreSaber leaderboard):{:,}".format(df_ranked_songs_from_leaderboard["Song"].count()))

    # if (total_count_from_leaderboard != df_ranked_songs_from_leaderboard["Song"].count()) and (err_scoresaber_ranked_songs_min <= total_count_from_leaderboard < err_scoresaber_ranked_songs_max):
    #     print("譜面数が一致していません")
    #     _ranked_song_from_leaderboard_is_enable = False

    # if total_count_from_leaderboard == df_ranked_songs_from_leaderboard["Song"].count():
    #     ranked_song_from_leaderboard_is_enable = True
    # else:
    #     print('ranked songs count is unmached. something is trouble.')

else:
    print("ranked songs count is mached. ok.")

<IPython.core.display.Javascript object>

LeaderBoard Ranked Song:5,291, Ranked Page:378
ranked songs count is mached. ok.


In [ ]:
#@title Get Player Scores Data
#@markdown Players' Score Data from ScoreSaber Public API - [doc](https://docs.scoresaber.com/)

def func_mode(x):
    if  x == "SoloStandard":
        return "Standard"
    else:
        return x

def func_score(x):
    if  x > 100:
        return "-"
    elif x == 100:
        return "SSS"
    elif x >= ss_plus_val and ss_plus_is_enable:
        return ss_plus
    elif x >= 90:
        return "SS"
    elif x >= 80:
        return "S"
    elif x >= 65:
        return "A"
    elif x >= 50:
        return "B"
    elif x >= 35:
        return "C"
    elif x >= 20:
        return "D"
    elif x >= 0:
        return "E"
    else:
        return "-"

def func_fc(x):
    if  x:
        return "FC"
    else:
        return "-"

def func_latest(x):
    if  x <= latest:
        return 1
    else:
        return 0

# url = r"https://scoresaber.com/api/player/{}/scores?sort=recent&limit={}".format(player_id, page_count)
# response = requests.get(url)
# res_data = response.json()
# df_scores = json_normalize(res_data['playerScores'])

# for i in tqdm(range(2, RangeCount)):
#     url = r"https://scoresaber.com/api/player/{}/scores?sort=recent&page={}&limit={}".format(player_id, i, page_count)
#     try:
#         response = requests.get(url)
#         res_data = response.json()
#         df_scores=df_scores.append(json_normalize(res_data['playerScores']), ignore_index=True)
#     except:
#         break

if saved_player_score_is_enable and os.path.exists(player_score_pickle_path):
    df_scores_pkl = pd.read_pickle(player_score_pickle_path)
    _df_scores_pkl = df_scores_pkl.head(0)

    i = 1
    while True:
        url = r"https://scoresaber.com/api/player/{}/scores?sort=recent&page={}&limit={}".format(player_id, i, page_count)
        try:
            response = requests.get(url)
            res_data = response.json()
            _df_scores_pkl=_df_scores_pkl.append(json_normalize(res_data['playerScores']), ignore_index=True)
            if df_scores_pkl['score.timeSet'].max() > _df_scores_pkl['score.timeSet'].min():
                break
        except:
            break

    df_scores=df_scores_pkl.append(_df_scores_pkl, ignore_index=True).sort_values("score.timeSet", ascending=False).groupby("score.id").head(1)

else:
    url = r"https://scoresaber.com/api/player/{}/scores?sort=recent&limit={}".format(player_id, page_count)
    response = requests.get(url)
    res_data = response.json()
    df_scores = json_normalize(res_data['playerScores'])

    for i in tqdm(range(2, RangeCount)):
        url = r"https://scoresaber.com/api/player/{}/scores?sort=recent&page={}&limit={}".format(player_id, i, page_count)
        try:
            response = requests.get(url)
            res_data = response.json()
            df_scores=df_scores.append(json_normalize(res_data['playerScores']), ignore_index=True)
        except:
            break

# 未加工データ保存
df_scores.to_pickle(player_score_pickle_path)

# 加工
df_scores['Song'] = df_scores['leaderboard.songName'] + " " + df_scores['leaderboard.songSubName'] + " / " + df_scores['leaderboard.songAuthorName'] + " [" + df_scores['leaderboard.levelAuthorName'] + "]"
df_scores['SongName'] = df_scores['leaderboard.songName']
df_scores['SongSub'] = df_scores['leaderboard.songSubName']
df_scores['SongAuthor'] = df_scores['leaderboard.songAuthorName']
df_scores['LevelAuthor'] = df_scores['leaderboard.levelAuthorName']
df_scores['Hash'] = df_scores['leaderboard.songHash'].str.upper()

def func_higher_score(x):
    try:
        if x[0] >= x[1]:
            return x[0]
        else:
            return x[1]
    except:
        return x[1]

if score_mode == "BaseScore":
    df_scores['Score'] = df_scores['score.baseScore']
elif score_mode == "ModifiedScore":
    df_scores['Score'] = df_scores['score.modifiedScore']
elif score_mode == "HigherScore":
    df_scores['Score'] = df_scores[['score.baseScore','score.modifiedScore']].apply(func_higher_score, axis=1)
else:
    df_scores['Score'] = df_scores['score.modifiedScore']

df_scores['Acc'] = df_scores['Score'] / df_scores['leaderboard.maxScore'] * 100
# df_scores['Acc'] = df_scores['score.modifiedScore'] / df_scores['leaderboard.maxScore'] * 100
df_scores['MaxScore'] = df_scores['leaderboard.maxScore']

df_scores['Mode'] = df_scores['leaderboard.difficulty.gameMode'].apply(func_mode)
#df_scores['Mode'] = df_scores['leaderboard.difficulty.gameMode']
_df_scores = df_scores['leaderboard.difficulty.difficultyRaw'].str.split('_', expand=True)
_df_scores.columns = ['_','Difficulty', 'Mode']
df_scores['Difficulty'] = _df_scores['Difficulty']
df_scores['Stars'] = df_scores['leaderboard.stars']
df_scores['Level'] = df_scores['Stars'].astype('int')
df_scores["LevelStr"] = df_scores['Level'].astype('str')
# df_scores['Score'] = df_scores['score.modifiedScore']
df_scores['Bad'] = df_scores['score.badCuts']
df_scores['Miss'] = df_scores['score.missedNotes']
df_scores['Combo'] = df_scores['score.maxCombo']
df_scores['PP'] = df_scores['score.pp']
df_scores['PPWeight'] = df_scores['score.pp'] * df_scores['score.weight']
df_scores['Rank'] = df_scores['score.rank']
df_scores['Modifiers'] = df_scores['score.modifiers']
df_scores['Ranked'] = df_scores['leaderboard.ranked']
df_scores['Qualified'] = df_scores['leaderboard.qualified']
df_scores['Play'] = df_scores['leaderboard.plays']
df_scores['DailyPlay'] = df_scores['leaderboard.dailyPlays']
df_scores['DateUtc'] = pd.to_datetime(df_scores['score.timeSet'])
_df_scores_idx = df_scores.set_index('DateUtc')
df_scores['DateJa'] = _df_scores_idx.index.tz_convert(timezone)
df_scores['Date'] = df_scores['DateJa'].dt.date
df_scores['Days'] = (tz_ja.date() - df_scores['Date']).dt.days
df_scores = df_scores.set_index('DateJa')
df_scores['Months'] = (df_scores['Days'] / 30).astype('int')
df_scores['DaysStr'] = df_scores['Days'].astype('str')
df_scores['MonthsStr'] = df_scores['Months'].astype('str')

df_scores['Latest'] = df_scores['Days'].apply(func_latest)

df_scores['AccRank'] = df_scores['Acc'].apply(func_score)

df_scores['Cover'] = '<img src="'+df_scores['leaderboard.coverImage']+'" style="width:{}px;"/>'.format(cover_image_size)

df_scores['FC'] = df_scores['score.fullCombo'].apply(func_fc)

df_scores = df_scores[[x for x in df_scores.columns if not x.startswith("score.")]]
df_scores = df_scores[[x for x in df_scores.columns if not x.startswith("leaderboard.")]]


# 結合前Score情報の抽出(LevelClearedProgress用)
df_scores_org = df_scores.copy()

# 改行コード等の除去
for col in df_scores.columns:
    try:
        if len(df_scores[df_scores[col].str.contains("\n")][[col]]) == 0:
            continue
        else:
            df_scores[col] = df_scores[col].str.replace("\n","")
    except:
        continue

for col in df_scores.columns:
    try:
        if len(df_scores[df_scores[col].str.contains("\r")][[col]]) == 0:
            continue
        else:
            df_scores[col] = df_scores[col].str.replace("\r","")
    except:
        continue


# RankedMap(らっきょさんのBeatSaverデータ)の情報結合
df_scores = df_scores.reset_index()
#df_scores = pd.merge(df_scores, df_rankmap_data[cols_rankedmap], on=["Hash", "Difficulty"], how="left", suffixes=("", "_y"))
df_scores = pd.merge(df_scores, df_rankmap_data, on=["Hash", "Difficulty"], how="left", suffixes=("", "_y"))
df_scores = df_scores[[x for x in df_scores.columns if not x.endswith("_y")]]
df_scores = df_scores.set_index("DateJa")
# df_scores = df_scores.drop('index', axis=1).drop_duplicates()

#Score情報の保存
df_scores[(df_scores['Ranked'] == True)][cols_score].sort_index(ascending=False).to_csv(player_ranked_path)
df_scores = df_scores[(df_scores['Ranked'] == True)].sort_index(ascending=False)
df_scores.sort_index(ascending=False).to_csv(player_score_path.format(player_id))

print('ScoreCount:{}, LatestPlayCount:{}, RankedPlayCount:{}'.format(df_scores['Play'].count(), df_scores[(df_scores['Latest'] == 1)]['Play'].count(), df_scores[
                                    (df_scores['Ranked']==1)
                                    & (df_scores['Mode']=="Standard")]['Play'].count()))

# ---------------------------------------
#@title Acc再計算
# ---------------------------------------
#@markdown ---
#@markdown <h4>Visible</h4>
recalq_visible = False #@param {type:"boolean"}

def func_max_score(x):
    combo_a = 115 * 1
    combo_b = 115 * 2
    combo_c = 115 * 4
    combo_d = 115 * 8

    if  x >= 14:
        return combo_d * (x - 13) + combo_a + combo_b * 4 + combo_c * 8
    elif x >= 6:
        return combo_c * (x - 5) + combo_a + combo_b * 4
    elif x >= 2:
        return combo_b * (x - 1) + combo_a
    elif x >= 1:
        return combo_a
    else:
        return 0

df_scores['MaxScore'] = df_scores['MaxScore']
df_scores['MaxScoreRecalq'] = df_scores['Notes'].apply(func_max_score)
df_scores['AccRecalq'] = df_scores['Score'] / df_scores['MaxScoreRecalq'] * 100
df_scores['AccRankRecalq'] = df_scores['AccRecalq'].apply(func_score)
df_scores['MaxScoreDiff'] = df_scores['MaxScore'].fillna(0) - df_scores['MaxScoreRecalq'].fillna(0)
df_scores['AccDiff'] = df_scores['Acc'].fillna(0) - df_scores['AccRecalq'].fillna(0)

df_errors = df_scores[
            (1==1)
            &(df_scores['Ranked'])
            &(df_scores['MaxScoreDiff'] != 0)
            ]

print('There are {} results where Acc is different from the game.'.format(len(df_errors)))
#print('Accがゲームと異なる結果が{}件あります。'.format(len(df_errors)))

if acc_recalq_override_is_enable:
    df_scores.rename(columns={
                            "MaxScore":"MaxScoreOrg",
                             "Acc":"AccOrg",
                             "AccRank":"AccRankOrg"
                             }, inplace=True)
    df_scores.rename(columns={
                            "MaxScoreRecalq":"MaxScore",
                             "AccRecalq":"Acc",
                             "AccRankRecalq":"AccRank"
                             }, inplace=True)
    print('Use the result of recalculating Acc.')

if recalq_visible:
    print('The following are the results extracted during recalculation.')
else:
    df_errors = df_errors.head(0)
    #print('表示設定は無効です。')

df_errors[cols_recalq].style.set_table_styles(styles_data).applymap(
    color_negative_red, subset=["AccDiff","MaxScoreDiff"]
        ).applymap(
            color_difficulty, subset=["Difficulty"]
        ).applymap(
            color_acc_rank, subset=["AccRank","AccRankRecalq"]
        # ).applymap(
        #     color_fc, subset=["FC"]
        ).format(
            style_format, na_rep="-"
        )

<IPython.core.display.Javascript object>

100%|██████████| 53/53 [01:17<00:00,  1.46s/it]


ScoreCount:4410, LatestPlayCount:0, RankedPlayCount:4410
There are 0 results where Acc is different from the game.
Use the result of recalculating Acc.


,Hash,Cover,SongName,Difficulty,Stars,Notes,Acc,AccRecalq,AccDiff,AccRank,AccRankRecalq,MaxScore,MaxScoreRecalq,MaxScoreDiff,Score,Miss,Combo,Preview
DateJa,,,,,,,,,,,,,,,,,,


In [ ]:
#@title Get ScoreSaberRanking Data
#@markdown Various ScoreSaber Rankings Data from [ScoreSaberRanking](https://github.com/rynan4818/ScoreSaberRanking) by [rynan4818](https://twitter.com/rynan4818)


# 古い処理---------------------
old_format_is_enable = False  #param {type:"boolean"}
try:
    if old_format_is_enable:
        user_name = df_info["name"][0]
        def get_your_gl_rank(name, url):
            _df_your_rank = pd.read_html(url)[0]
            df_info_rank = df_info.copy()
            try:
                your_rank = _df_your_rank[_df_your_rank['ユーザー名']==user_name]["順位"].values[0]
                your_index = _df_your_rank[_df_your_rank['ユーザー名']==user_name].index[0]
                your_val = _df_your_rank.iloc[your_index,4]
                next_name = _df_your_rank.iloc[your_index-1,1]
                next_countory = _df_your_rank.iloc[your_index-1,2]
                next_pp = _df_your_rank.iloc[your_index-1,3]
                next_val = _df_your_rank.iloc[your_index-1,4]
                if your_rank > 1:
                    print("{} : #{:,}, {} | next:{} by {}[{}]-{}".format(name, your_rank, your_val, next_val, next_name, next_countory, next_pp))
                else:
                    print("{} : #{:,}, {} ☆☆☆☆☆☆☆☆☆☆☆".format(name, your_rank, your_val))
            except:
                your_rank = 99999
            return your_rank

        def get_your_jp_rank(name, url):
            _df_your_rank = pd.read_html(url)[0]
            df_info_rank = df_info.copy()
            try:
                your_rank = _df_your_rank[_df_your_rank['ユーザー名']==user_name]["順位"].values[0]
                your_index = _df_your_rank[_df_your_rank['ユーザー名']==user_name].index[0]
                your_val = _df_your_rank.iloc[your_index,3]
                next_name = _df_your_rank.iloc[your_index-1,1]
                next_pp = _df_your_rank.iloc[your_index-1,2]
                next_val = _df_your_rank.iloc[your_index-1,3]
                if your_rank > 1:
                    print("{} : #{:,}, {} | next:{} by {}-{}".format(name, your_rank, your_val, next_val, next_name, next_pp))
                else:
                    print("{} : #{:,}, {} <<<<<<<<<<<<<<<<<<<<<<<<< No.1".format(name, your_rank, your_val))
            except:
                your_rank = 99999
            #print("{} : #{:,}".format(name, your_rank))
            return your_rank

        def get_your_country_rank(name, country, url):
            _df_your_rank = pd.read_html(url)[0]
            df_info_rank = df_info.copy()
            try:
                _df_your_rank['国']=_df_your_rank['国'].str.upper()
                _df_your_country = _df_your_rank[_df_your_rank['国']==country].reset_index().drop('index', axis=1)
                _df_your_country['順位'] = _df_your_country.index + 1

                your_rank = _df_your_country[_df_your_country['ユーザー名']==user_name]["順位"].values[0]
                #your_rank = _df_your_rank[_df_your_rank['ユーザー名']==user_name].index[0]
                your_index = _df_your_country[_df_your_country['ユーザー名']==user_name].index[0]
                your_val = _df_your_country.iloc[your_index, 4]
                next_name = _df_your_country.iloc[your_index-1, 1]
                next_countory = _df_your_country.iloc[your_index-1, 2]
                next_pp = _df_your_country.iloc[your_index-1, 3]
                next_val = _df_your_country.iloc[your_index-1, 4]
                if your_rank > 1:
                    print("{} : #{:,}, {} | next:{} by {}[{}]-{}".format(name, your_rank, your_val, next_val, next_name, next_countory, next_pp))
                else:
                    print("{} : #{:,}, {} ☆☆☆☆☆☆☆☆☆☆☆".format(name, your_rank, your_val))
            except:
                your_rank = 99999
            #print("{} : #{:,}".format(name, your_rank))
            return your_rank

        url_total_play_count = "https://github.com/rynan4818/ScoreSaberRanking/blob/master/total_play_count_global_rank.md"
        url_total_play_count_jp = "https://github.com/rynan4818/ScoreSaberRanking/blob/master/total_play_count_jp_rank.md"
        url_ranked_play_count = "https://github.com/rynan4818/ScoreSaberRanking/blob/master/ranked_play_count_global_rank.md"
        url_ranked_play_count_jp = "https://github.com/rynan4818/ScoreSaberRanking/blob/master/ranked_play_count_jp_rank.md"
        url_total_score = "https://github.com/rynan4818/ScoreSaberRanking/blob/master/total_score_global_rank.md"
        url_total_score_jp = "https://github.com/rynan4818/ScoreSaberRanking/blob/master/total_score_jp_rank.md"
        url_ranked_score = "https://github.com/rynan4818/ScoreSaberRanking/blob/master/total_ranked_score_global_rank.md"
        url_ranked_score_jp = "https://github.com/rynan4818/ScoreSaberRanking/blob/master/total_ranked_score_jp_rank.md"
        url_ave_ranked_acc = "https://github.com/rynan4818/ScoreSaberRanking/blob/master/average_ranked_accuracy_global_rank.md"
        url_ave_ranked_acc_jp = "https://github.com/rynan4818/ScoreSaberRanking/blob/master/average_ranked_accuracy_jp_rank.md"

        df_info["TotalPlayRank"] = get_your_gl_rank("Total Play Count Global Rank", url_total_play_count)#, df_info_rank)
        df_info["TotalPlayJPRank"] = get_your_country_rank("Total Play Count {} Rank".format(df_info["country"][0]), df_info["country"][0], url_total_play_count)
        df_info["RankedPlayRank"] = get_your_gl_rank("Ranked Play Count Global Rank", url_ranked_play_count)#, df_info_rank)
        df_info["RankedPlayJPRank"] = get_your_country_rank("Ranked Play Count {} Rank".format(df_info["country"][0]), df_info["country"][0], url_ranked_play_count)
        df_info["TotalScoreRank"] = get_your_gl_rank("Total Score Global Rank", url_total_score)#, df_info_rank)
        df_info["TotalScoreJPRank"] = get_your_country_rank("Total Score {} Rank".format(df_info["country"][0]), df_info["country"][0], url_total_score)
        df_info["RankedScoreRank"] = get_your_gl_rank("Ranked Score Global Rank", url_ranked_score)#, df_info_rank)
        df_info["RankedScoreJPRank"] = get_your_country_rank("Ranked Score {} Rank".format(df_info["country"][0]), df_info["country"][0], url_ranked_score)
        df_info["AveRankedAccRank"] = get_your_gl_rank("Average Ranked Accuracy Global Rank", url_ave_ranked_acc)#, df_info_rank)
        df_info["AveRankedAccJPRank"] = get_your_country_rank("Average Ranked Accuracy {} Rank".format(df_info["country"][0]), df_info["country"][0], url_ave_ranked_acc)
        # # 古い処理---------------------


    #@title  ScoreSaberRanking 情報の取得
    import requests
    import json

    # ステップ1: 指定のURLからJSONを取得します。
    url_index = "https://rynan4818.github.io/ScoreSaberRanking/json/scoresaber_rank_index.json"
    response = requests.get(url_index)
    data_index = json.loads(response.text)

    # ScoreSaber UserID (ここでは例として "76561198333869741")
    user_id = str(player_id)

    # print(data_index)

    # ステップ2: 該当のScoreSaber UserIDのRankingDataFile配列のインデックスを取得し、そのインデックスで該当する"RankingDataFile"を取得します。
    ranking_data_file_index = data_index["UserIndexData"][user_id][0]
    ranking_data_file = data_index["RankingDataFile"][ranking_data_file_index]

    # ステップ3: 以下のurl_rankingを生成します。
    url_ranking = f"https://rynan4818.github.io/ScoreSaberRanking/json/{ranking_data_file}"

    # url_ranking から必要なデータを取得します。
    response_ranking = requests.get(url_ranking)
    data_ranking = json.loads(response_ranking.text)

    # print(data_ranking)

    # -----------------------
    # データの格納
    # -----------------------
    import pandas as pd

    # ステップ1: Columnを列名、UserDataをレコードとして、データフレームにデータを格納します。
    df = pd.DataFrame(data_ranking['UserData'], columns=data_ranking['Column'])

    # ステップ2: データフレームの該当のUserIDのレコードを抽出します。
    user_record = df[df['UserID'] == int(user_id)]

    user_record = user_record.reset_index()
    df_info["TotalPlayRank"] = user_record['TotalPlayCountRank'][0]
    df_info["TotalPlayJPRank"] = user_record['TotalPlayCountLocalRank'][0]
    df_info["RankedPlayRank"] = user_record['RankedPlayCountRank'][0]
    df_info["RankedPlayJPRank"] = user_record['RankedPlayCountLocalRank'][0]

    df_info["TotalScoreRank"] = user_record['TotalScoreRank'][0]
    df_info["TotalScoreJPRank"] = user_record['TotalScoreLocalRank'][0]
    df_info["RankedScoreRank"] = user_record['TotalRankedScoreRank'][0]
    df_info["RankedScoreJPRank"] = user_record['TotalRankedScoreLocalRank'][0]

    df_info["AveRankedAccRank"] = user_record['AverageRankedAccuracyRank'][0]
    df_info["AveRankedAccJPRank"] = user_record['AverageRankedAccuracyLocalRank'][0]

    # -----------------------
    # データの表示用の成型と表示
    # -----------------------

    # 抽出したい列のリスト
    columns_to_extract = [
        'TotalScoreRank',
        'TotalScoreRankNextUser',
        'TotalScoreLocalRank',
        'TotalScoreLocalRankNextUser',
        'TotalScoreWeeklyChangeValue',
        'TotalScoreMonthlyChangeValue',
        'TotalRankedScoreRank',
        'TotalRankedScoreRankNextUser',
        'TotalRankedScoreLocalRank',
        'TotalRankedScoreLocalRankNextUser',
        'TotalRankedScoreWeeklyChangeValue',
        'TotalRankedScoreMonthlyChangeValue',
        'AverageRankedAccuracyRank',
        'AverageRankedAccuracyRankNextUser',
        'AverageRankedAccuracyLocalRank',
        'AverageRankedAccuracyLocalRankNextUser',
        'AverageRankedAccuracyWeeklyChangeValue',
        'AverageRankedAccuracyMonthlyChangeValue',
        'TotalPlayCountRank',
        'TotalPlayCountRankNextUser',
        'TotalPlayCountLocalRank',
        'TotalPlayCountLocalRankNextUser',
        'TotalPlayCountWeeklyChangeValue',
        'TotalPlayCountMonthlyChangeValue',
        'RankedPlayCountRank',
        'RankedPlayCountRankNextUser',
        'RankedPlayCountLocalRank',
        'RankedPlayCountLocalRankNextUser',
        'RankedPlayCountWeeklyChangeValue',
        'RankedPlayCountMonthlyChangeValue',
        'ReplaysWatchedRank',
        'ReplaysWatchedRankNextUser',
        'ReplaysWatchedLocalRank',
        'ReplaysWatchedLocalRankNextUser',
        'ReplaysWatchedWeeklyChangeValue',
        'ReplaysWatchedMonthlyChangeValue',
        'WeeklyChangeRank',
        'WeeklyChangeRankNextUser',
        'WeeklyChangeLocalRank',
        'WeeklyChangeLocalRankNextUser',
        'WeeklyChangeWeeklyChangeValue',
        'WeeklyChangeMonthlyChangeValue'
    ]

    #-----------------------------
    # 特定の列を抽出
    extracted_record = user_record[columns_to_extract]

    # 同一の構造を持つ部分をまとめて表示
    grouped_columns = [
        'Rank',
        'RankNextUser',
        'LocalRank',
        'LocalRankNextUser',
        'WeeklyChangeValue',
        'MonthlyChangeValue'
    ]

    # 各カテゴリに分けてデータを整形
    formatted_data = []
    for i in range(0, len(columns_to_extract), len(grouped_columns)):
        row = extracted_record.iloc[:, i:i+len(grouped_columns)].values.flatten().tolist()
        row.insert(0, columns_to_extract[i].rstrip(grouped_columns[0]))  # カテゴリ名を追加
        formatted_data.append(row)

    # 表形式で表示
    df_formatted = pd.DataFrame(formatted_data, columns=['Category'] + grouped_columns)
    # print(df_formatted)

    # 数値の列を整数に変換
    int_columns = ['Rank', 'RankNextUser', 'LocalRank', 'LocalRankNextUser', 'WeeklyChangeValue', 'MonthlyChangeValue']
    # df_formatted[int_columns] = df_formatted[int_columns].astype(int)


    #-----------------------------

    # データフレーム df から scoreStats を含む列を抽出
    filtered_df = df_info.filter(like='scoreStats')

    # 列名をリネームするための関数
    def rename_columns(col_name):
        col_name = col_name.replace('scoreStats.', '')  # "scoreStats." を削除
        return col_name[0].upper() + col_name[1:]  # 最初の文字だけを大文字に

    # 列名をリネーム
    renamed_df = filtered_df.rename(columns=rename_columns)

    #-----------------------------

    # 数値を特定のフォーマットの文字列に変換する関数
    def format_value(value):
        if pd.isnull(value):  # 値が null の場合はそのまま返す
            return None
        elif value.is_integer():  # 整数を表す場合（1.0 を含む）
            return '{:,}'.format(int(value))  # value を整数に変換してから文字列に変換
        else:  # 小数点を含む場合は小数第2位までにし、%を付ける
            return '{:.2f}%'.format(value)  # 値をパーセントに変換

    # カテゴリ列と一致する列名の値を取得し、それを文字列に変換する関数
    def get_value(row):
        category = row['Category']
        if category in renamed_df.columns:
            value = renamed_df[category].values[0]
            return format_value(value)  # format_value() 関数で値を文字列に変換
        else:
            return None

    # Value 列を追加
    # df['Value'] = df.apply(get_value, axis=1)

    df_formatted['Value'] = df_formatted.apply(get_value, axis=1)

    style_ranking = {
            "Rank": "#{:,.0f}",
            "LocalRank": "#{:,.0f}",
            # "Value": "#{:,.2f}",
        }

    formatted_columns = ['Category','Value','Rank', 'LocalRank']

    print("")

    display(df_formatted[formatted_columns].style.highlight_null(null_color="lightgray" # ).format(style_format)
        ).format(style_ranking).set_table_attributes("style='display:inline'"
        # ).set_caption("Summary - {} - {}".format(tz_ja.strftime("%Y.%m.%d"), df_info["name"][0])
        ))#.set_table_styles(caption_styles)
except:
    print('request error is occur.')
    df_info["TotalPlayRank"] = 99999
    df_info["TotalPlayJPRank"] = 99999
    df_info["RankedPlayRank"] = 99999
    df_info["RankedPlayJPRank"] = 99999

    df_info["TotalScoreRank"] = 99999
    df_info["TotalScoreJPRank"] = 99999
    df_info["RankedScoreRank"] = 99999
    df_info["RankedScoreJPRank"] = 99999

    df_info["AveRankedAccRank"] = 99999
    df_info["AveRankedAccJPRank"] = 99999

<IPython.core.display.Javascript object>

,Category,Value,Rank,LocalRank
0,TotalScore,"4,017,954,293",#241,#85
1,TotalRankedScore,"3,403,385,314",#13,#9
2,AverageRankedAccuracy,92.67%,"#1,111",#60
3,TotalPlayCount,"5,304",#209,#87
4,RankedPlayCount,"4,410",#8,#5
5,ReplaysWatched,47,#436,#31
6,WeeklyChange,None,"#6,148",#592


# PreProcess

In [ ]:
#@title Calc Level Cleared Progress

if ranked_song_from_leaderboard_is_enable:
    df_progress = df_ranked_songs_from_leaderboard[["Level","Song"]].groupby(["Level"],as_index=False).count()
else:
    df_progress = df_rankmap_data[["Level","Song"]].fillna('Unknown').groupby(["Level"],as_index=False).count()

# Level別Clear数(No Fail除く)
df_cleared = df_scores[(1==1)
                &(df_scores["Modifiers"] != "NF")
                # & (df_scores["Ranked"] == True)
                ][["Level","Song"]].groupby(["Level"],as_index=False).count().rename(columns={"Song": "Cleared"})

# Level別 No fail数
df_cleared_nf = df_scores[(1==1)
                &(df_scores["Modifiers"] == "NF")
                # & (df_scores["Ranked"] == True)
                ][["Level","Song"]].groupby(["Level"],as_index=False).count().rename(columns={"Song": "NF"})

# 直近のLevel別Clear数(No Fail除く)
df_cleared_latest = df_scores[(1==1)
                       &(df_scores["Modifiers"] != "NF")
                    #    & (df_scores['Ranked'] == True)
                       & (df_scores["Latest"] == 1)
                ][["Level","Song"]].groupby(["Level"],as_index=False).count().rename(columns={"Song": "RecentCleared"})

# 結合
df_progress = pd.merge(df_progress, df_cleared_latest, on="Level", how="outer")
df_progress = pd.merge(df_progress, df_cleared, on="Level", how="outer")
df_progress = pd.merge(df_progress, df_cleared_nf, on="Level", how="outer")


def func_byplayed(x):
    if x[0]> 0:
        return x[0]/x[1] * 100
    else:
        return 0


df_progress["NotCleared"] = df_progress["Song"].fillna(0) - df_progress["Cleared"].fillna(0) - df_progress["NF"].fillna(0)
df_progress["AlreadyCleared"] = df_progress["Cleared"].fillna(0) - df_progress["RecentCleared"].fillna(0)
df_progress["Played"] = df_progress["Cleared"].fillna(0) + df_progress["NF"].fillna(0)
df_progress = df_progress.fillna(0)
df_progress["NotClearedRate"] = df_progress["NotCleared"] / df_progress["Song"] * 100
df_progress["AlreadyClearedRate"] = df_progress["AlreadyCleared"] / df_progress["Song"] * 100
df_progress["RecentClearedRate"] = df_progress["RecentCleared"] / df_progress["Song"] * 100
df_progress["NFRate"] = df_progress["NF"] / df_progress["Song"] * 100

## プレイ追加
df_progress["AlreadyClearedRateByPlayed"] = df_progress[["AlreadyCleared","Played"]].apply(func_byplayed, axis=1)
df_progress["RecentClearedRateByPlayed"] = df_progress[["RecentCleared","Played"]].apply(func_byplayed, axis=1)
df_progress["NFRateByPlayed"] = df_progress[["NF","Played"]].apply(func_byplayed, axis=1)


# Accの内訳
df_acc =df_scores[(1==1)
                &(df_scores["Modifiers"] != "NF")
                # & (df_scores['Ranked'] == True)
                ].pivot_table(index="Level", columns= "AccRank", values="Song", aggfunc="count").fillna(0).reset_index()

if ss_plus_is_enable:
    cols_accs = ["SSS",ss_plus,"SS","S","A","B"]
else:
    cols_accs = ["SSS","SS","S","A","B"]

for cols_acc in cols_accs:
    if cols_acc in df_acc.columns :
        pass
    else:
        print("Rank {}is nothing.".format(cols_acc))
        df_acc[cols_acc] = 0

# Accの結合
df_progress = pd.merge(df_progress, df_acc, on="Level", how="outer").fillna(0)
df_progress["Other"] = df_progress["Song"] - df_progress["SSS"] - df_progress["SS"] - df_progress["S"] - df_progress["A"] - df_progress["B"]
df_progress["SSS-Rate"] = df_progress["SSS"] / df_progress["Song"] * 100
if ss_plus_is_enable:
    df_progress["Other"] = df_progress["Other"] - df_progress[ss_plus]
    df_progress[ss_plus_rate] = df_progress[ss_plus] / df_progress["Song"] * 100
df_progress["SS-Rate"] = df_progress["SS"] / df_progress["Song"] * 100
df_progress["S-Rate"] = df_progress["S"] / df_progress["Song"] * 100
df_progress["A-Rate"] = df_progress["A"] / df_progress["Song"] * 100
df_progress["B-Rate"] = df_progress["B"] / df_progress["Song"] * 100
df_progress["Other-Rate"] = df_progress["Other"] / df_progress["Song"] * 100

## プレイ追加
df_progress["OtherPlayed"] = df_progress["Played"] - df_progress["SSS"] - df_progress["SS"] - df_progress["S"] - df_progress["A"] - df_progress["B"]
df_progress["SSS-RateByPlayed"] = df_progress[["SSS","Played"]].apply(func_byplayed, axis=1)
if ss_plus_is_enable:
    df_progress["OtherPlayed"] = df_progress["OtherPlayed"] - df_progress[ss_plus]
    df_progress[ss_plus_rate+"ByPlayed"] = df_progress[[ss_plus,"Played"]].apply(func_byplayed, axis=1)
df_progress["SS-RateByPlayed"] = df_progress[["SS","Played"]].apply(func_byplayed, axis=1)
df_progress["S-RateByPlayed"] = df_progress[["S","Played"]].apply(func_byplayed, axis=1)
df_progress["A-RateByPlayed"] = df_progress[["A","Played"]].apply(func_byplayed, axis=1)
df_progress["B-RateByPlayed"] = df_progress[["B","Played"]].apply(func_byplayed, axis=1)
df_progress["Other-RateByPlayed"] = df_progress[["OtherPlayed","Played"]].apply(func_byplayed, axis=1)


# FCの内訳
df_fc = df_scores[(1==1)
                &(df_scores["Modifiers"] != "NF")
                # & (df_scores["Ranked"] == True)
                & (df_scores["FC"] == "FC")
                ][["Level","Song"]].groupby(["Level"],as_index=False).count().rename(columns={"Song": "FC"})

df_fc_latest = df_scores[(1==1)
            &(df_scores["Modifiers"] != "NF")
            # & (df_scores['Ranked'] == True)
            & (df_scores["Latest"] == 1)
            & (df_scores["FC"] == "FC")
        ][["Level","Song"]].groupby(["Level"],as_index=False).count().rename(columns={"Song": "RecentFC"})

# FCの結合
df_progress = pd.merge(df_progress, df_fc, on="Level", how="outer")
df_progress = pd.merge(df_progress, df_fc_latest, on="Level", how="outer")
df_progress["AlreadyFC"] = df_progress["FC"].fillna(0) - df_progress["RecentFC"].fillna(0)
df_progress["NotFC"] = df_progress["Cleared"].fillna(0) + df_progress["NF"].fillna(0) - df_progress["FC"].fillna(0)
df_progress = df_progress.fillna(0)
df_progress["AlreadyFCRate"] = df_progress["AlreadyFC"] / df_progress["Song"] * 100
df_progress["RecentFCRate"] = df_progress["RecentFC"] / df_progress["Song"] * 100
df_progress["NotFCRate"] = df_progress["NotFC"] / df_progress["Song"] * 100
## プレイ追加
df_progress["AlreadyFCRateByPlayed"] = df_progress[["AlreadyFC","Played"]].apply(func_byplayed, axis=1)
df_progress["RecentFCRateByPlayed"] = df_progress[["RecentFC","Played"]].apply(func_byplayed, axis=1)
df_progress["NotFCRateByPlayed"] = df_progress[["NotFC","Played"]].apply(func_byplayed, axis=1)

# プレイ追加によるnaの対応
df_progress.fillna(0)

df_progress.to_csv(level_cleared_path, index=False)

# 検算
ranked_count = df_progress["Song"].sum()
cleared_count = int(df_progress["Cleared"].sum())
nofail_count = int(df_progress["NF"].sum())
play_count = cleared_count + nofail_count
fc_count = int(df_progress["FC"].sum())
recent_cleared_count = int(df_progress["RecentCleared"].sum())

print("RankSongs:{:,} | ClearCount:{:,}, NF Count:{:,}, PlayCount:{:,}, FC Count:{:,}".format(ranked_count, cleared_count, nofail_count, play_count, fc_count))

<IPython.core.display.Javascript object>

Rank SSSis nothing.
RankSongs:5,291 | ClearCount:4,336, NF Count:74, PlayCount:4,410, FC Count:2,213


In [ ]:
#@title Save Player Infomation

#@title FC情報の結合 Player情報の記録と読込(df_infos)
df_info["TotalFC"] = df_scores_org[df_scores_org['FC']=='FC']['FC'].count().astype('int')
df_info["RankedFC"] = df_progress["FC"].sum().astype('int')
PlayCount = df_info["TotalPlay"][0]
RangeCount = math.ceil(PlayCount / page_count) + 1
print("TotalPlayCount:{:,}, PageCount:{:,}".format(PlayCount, RangeCount))

if os.path.exists(player_info_path):
    df_infos = pd.read_csv(player_info_path)
    df_info_csv = df_info[cols_info].copy()
    # 結合とソート
    df_info_csv = df_info_csv.append(df_infos).sort_values('ScoreDate').reset_index().drop('index', axis=1)
    df_info_csv.to_csv(player_info_path, index=None)
else:
    df_info[cols_info].to_csv(player_info_path, index=None)

df_infos = pd.read_csv(player_info_path)
df_infos["ScoreDate"] = pd.to_datetime(df_infos["ScoreDate"])

<IPython.core.display.Javascript object>

TotalPlayCount:5,304, PageCount:55


# Analysis

In [ ]:
#@title ## Player Summary
#@markdown ---
#@markdown <h4>Enable</h4>
player_summary_is_enable = True #@param {type:"boolean"}

#@markdown ---
#@markdown <h4>Setting</h4>
rank_yaxis_reversed_is_enable = True #@param {type:"boolean"}
span = "ALL" #@param ["ALL", "1Y", "6M", "3M", "50D", "1M", "1W", "Recent"]
before_target = "Recent" #@param ["Recent", "SpanStart"]
since_x_days_ago = latest + 1
#param {type:"number"}
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# plotly ファイル名設定
plot_name = "PlayerSummary"
config = {
    'toImageButtonOptions': {
        'format': 'png',
        'filename': "{}_{}_{}".format(plot_name, df_info["name"][0], tz_ja.strftime("%Y%m%d_%H%M%S")),
        'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
    }
}

def get_summary_diff(_df_summary, _col, reversed_is_enable=False):
    _after = df_info.iloc[0][_col]
    try:
        if before_target == "Recent":
            _before = _df_summary.iloc[-1-since_x_days_ago][_col]
        else:
            _before = _df_summary.iloc[0][_col]
    except:
        _before = _after

    if reversed_is_enable:
        _diff = _before - _after
    else:
        _diff = _after - _before
    return _before, _after, _diff

if player_summary_is_enable:
    df_summary = df_infos.copy()

    df_summary["ScoreDateUtc"] = pd.to_datetime(df_summary['ScoreDate'], utc=True)
    _df_summary_idx = df_summary.set_index("ScoreDateUtc")
    df_summary["ScoreDateJa"] = _df_summary_idx.index.tz_convert("Asia/Tokyo")
    df_summary = df_summary.set_index("ScoreDateJa").interpolate('bfill')

    if not span == "ALL":

        if span == "1Y":
            dt_from = tz_ja - timedelta(days=366)
        elif span == "6M":
            dt_from = tz_ja - timedelta(days=181)
        elif span == "3M":
            dt_from = tz_ja - timedelta(days=91)
        elif span == "50D":
            dt_from = tz_ja - timedelta(days=51)
        elif span == "1M":
            dt_from = tz_ja - timedelta(days=32)
        elif span == "1W":
            dt_from = tz_ja - timedelta(days=8)
        elif span == "Recent":
            dt_from = tz_ja - timedelta(days=since_x_days_ago+1)

        str_from = dt_from.strftime('%Y/%m/%d')
        df_summary = df_summary[str_from:]

    df_summary = df_summary.reset_index()
    value_to_replace = 99999
    df_summary = df_summary.replace(value_to_replace, np.nan)
    df_summary = df_summary.fillna(method='ffill')

    df_summary_score = df_summary.set_index('ScoreDateJa').resample('1D').max().fillna(method='ffill')
    df_summary_rank = df_summary.set_index('ScoreDateJa').resample('1D').min().fillna(method='ffill')

    if len(df_summary_score) >= since_x_days_ago+1:
        if before_target == "SpanStart":
            from_date = "[{}]".format(df_summary_score.index[0].strftime('%m.%d'))
        else:
            if since_x_days_ago == 1:
                from_date = "YTD"
            else:
                from_date = "[{}]".format(df_summary_score.index[-1-since_x_days_ago].strftime('%m.%d'))
        rank_diff = get_summary_diff(df_summary_rank, 'rank', rank_yaxis_reversed_is_enable)
        country_rank_diff = get_summary_diff(df_summary_rank, 'countryRank', rank_yaxis_reversed_is_enable)
        pp_diff = get_summary_diff(df_summary_score, 'pp')

        acc_diff = get_summary_diff(df_summary_score, 'AveRankedAcc')
        acc_rank_diff = get_summary_diff(df_summary_score, 'AveRankedAccRank', rank_yaxis_reversed_is_enable)
        acc_jp_rank_diff = get_summary_diff(df_summary_score, 'AveRankedAccJPRank', rank_yaxis_reversed_is_enable)

        total_score_diff = get_summary_diff(df_summary_score, 'TotalScore')
        total_score_rank_diff = get_summary_diff(df_summary_score, 'TotalScoreRank', rank_yaxis_reversed_is_enable)
        total_score_jp_rank_diff = get_summary_diff(df_summary_score, 'TotalScoreJPRank', rank_yaxis_reversed_is_enable)

        ranked_score_diff = get_summary_diff(df_summary_score, 'RankedScore')
        ranked_score_rank_diff = get_summary_diff(df_summary_score, 'RankedScoreRank', rank_yaxis_reversed_is_enable)
        ranked_score_jp_rank_diff = get_summary_diff(df_summary_score, 'RankedScoreJPRank', rank_yaxis_reversed_is_enable)

        total_play_diff = get_summary_diff(df_summary_score, 'TotalPlay')
        total_play_rank_diff = get_summary_diff(df_summary_score, 'TotalPlayRank', rank_yaxis_reversed_is_enable)
        total_play_jp_rank_diff = get_summary_diff(df_summary_score, 'TotalPlayJPRank', rank_yaxis_reversed_is_enable)

        ranked_play_diff = get_summary_diff(df_summary_score, 'RankedPlay')
        ranked_play_rank_diff = get_summary_diff(df_summary_score, 'RankedPlayRank', rank_yaxis_reversed_is_enable)
        ranked_play_jp_rank_diff = get_summary_diff(df_summary_score, 'RankedPlayJPRank', rank_yaxis_reversed_is_enable)

        total_fc_diff = get_summary_diff(df_summary_score, 'TotalFC')
        ranked_fc_diff = get_summary_diff(df_summary_score, 'RankedFC')

        fig = make_subplots(
            rows=8, cols=3,
            horizontal_spacing=0.03,
            vertical_spacing=0.065,
            subplot_titles=(
                "Total PP : {} {:,.2f}->{:,.2f}({:+,.2f})".format(from_date,pp_diff[0],pp_diff[1],pp_diff[2]),
                "ScoreSaber - Global Rank : {} #{:,.0f}->#{:,.0f}({:+,.0f})".format(from_date,rank_diff[0],rank_diff[1],rank_diff[2]),
                "ScoreSaber - {} Rank : {} #{:,.0f}->#{:,.0f}({:+,.0f})".format(df_info["country"][0],from_date,country_rank_diff[0],country_rank_diff[1],country_rank_diff[2]),

                "Average Ranked Acc : {} {:,.2f}%->{:,.2f}%({:+,.2f}%)".format(from_date,acc_diff[0],acc_diff[1],acc_diff[2]),
                "Average Ranked Acc - Global Rank : {} #{:,.0f}->#{:,.0f}({:+,.0f})".format(from_date,acc_rank_diff[0],acc_rank_diff[1],acc_rank_diff[2]),
                "Average Ranked Acc - {} Rank : {} #{:,.0f}->#{:,.0f}({:+,.0f})".format(df_info["country"][0],from_date,acc_jp_rank_diff[0],acc_jp_rank_diff[1],acc_jp_rank_diff[2]),

                "Total Score : {} {:,.0f}->{:,.0f}({:+,.0f})".format(from_date,total_score_diff[0],total_score_diff[1],total_score_diff[2]),
                "Total Score - Global Rank : {} #{:,.0f}->#{:,.0f}({:+,.0f})".format(from_date,total_score_rank_diff[0],total_score_rank_diff[1],total_score_rank_diff[2]),
                "Total Score - {} Rank : {} #{:,.0f}->#{:,.0f}({:+,.0f})".format(df_info["country"][0],from_date,total_score_jp_rank_diff[0],total_score_jp_rank_diff[1],total_score_jp_rank_diff[2]),

                "Ranked Score : {} {:,.0f}->{:,.0f}({:+,.0f})".format(from_date,ranked_score_diff[0],ranked_score_diff[1],ranked_score_diff[2]),
                "Ranked Score - Global Rank : {} #{:,.0f}->#{:,.0f}({:+,.0f})".format(from_date,ranked_score_rank_diff[0],ranked_score_rank_diff[1],ranked_score_rank_diff[2]),
                "Ranked Score - {} Rank : {} #{:,.0f}->#{:,.0f}({:+,.0f})".format(df_info["country"][0],from_date,ranked_score_jp_rank_diff[0],ranked_score_jp_rank_diff[1],ranked_score_jp_rank_diff[2]),

                "Total Play Count : {} {:,.0f}->{:,.0f}({:+,.0f})".format(from_date,total_play_diff[0],total_play_diff[1],total_play_diff[2]),
                "Total Play Count - Global Rank : {} #{:,.0f}->#{:,.0f}({:+,.0f})".format(from_date,total_play_rank_diff[0],total_play_rank_diff[1],total_play_rank_diff[2]),
                "Total Play Count - {} Rank : {} #{:,.0f}->#{:,.0f}({:+,.0f})".format(df_info["country"][0],from_date,total_play_jp_rank_diff[0],total_play_jp_rank_diff[1],total_play_jp_rank_diff[2]),

                "Ranked Play Count : {} {:,.0f}->{:,.0f}({:+,.0f})".format(from_date,ranked_play_diff[0],ranked_play_diff[1],ranked_play_diff[2]),
                "Ranked Play Count - Global Rank : {} #{:,.0f}->#{:,.0f}({:+,.0f})".format(from_date,ranked_play_rank_diff[0],ranked_play_rank_diff[1],ranked_play_rank_diff[2]),
                "Ranked Play Count - {} Rank : {} #{:,.0f}->#{:,.0f}({:+,.0f})".format(df_info["country"][0],from_date,ranked_play_jp_rank_diff[0],ranked_play_jp_rank_diff[1],ranked_play_jp_rank_diff[2]),

                "Total FC Count : {} {:,.0f}->{:,.0f}({:+.0f})".format(from_date,total_fc_diff[0],total_fc_diff[1],total_fc_diff[2]),
                "","",
                "Ranked FC Count : {} {:,.0f}->{:,.0f}({:+,.0f})".format(from_date,ranked_fc_diff[0],ranked_fc_diff[1],ranked_fc_diff[2])
                ))

    else:
        fig = make_subplots(
            rows=8, cols=3,
            horizontal_spacing=0.03,
            vertical_spacing=0.05,
            subplot_titles=(
                "Total PP : {:,.2f}".format(df_info.iloc[0]['pp']),
                "Global Rank : #{:,}".format(df_info.iloc[0]['rank']),
                "{} Rank : #{:,}".format(df_info["country"][0],df_info.iloc[0]['countryRank']),

                "Average Ranked Acc : {:,.2f}%".format(df_info.iloc[0]['AveRankedAcc']),
                "Average Ranked Acc Rank: #{:,}".format(df_info.iloc[0]['AveRankedAccRank']),
                "Average Ranked Acc {} Rank: #{:,}".format(df_info["country"][0],df_info.iloc[0]['AveRankedAccJPRank']),

                "Total Score : {:,}".format(df_info.iloc[0]['TotalPlay']),
                "Total Score Rank : #{:,}".format(df_info.iloc[0]['TotalPlayRank']),
                "Total Score {} Rank : #{:,}".format(df_info["country"][0],df_info.iloc[0]['TotalPlayJPRank']),

                "Ranked Score : {:,}".format(df_info.iloc[0]['RankedPlay']),
                "Ranked Score Rank : #{:,}".format(df_info.iloc[0]['RankedScoreRank']),
                "Ranked Score {} Rank : #{:,}".format(df_info["country"][0],df_info.iloc[0]['RankedScoreJPRank']),

                "Total Play Count : {:,}".format(df_info.iloc[0]['TotalPlay']),
                "Total Play Count Rank : #{:,}".format(df_info.iloc[0]['TotalPlayRank']),
                "Total Play Count {} Rank : #{:,}".format(df_info["country"][0],df_info.iloc[0]['TotalPlayJPRank']),

                "Ranked Play Count : {:,}".format(df_info.iloc[0]['RankedPlay']),
                "Ranked Play Count Rank : #{:,}".format(df_info.iloc[0]['RankedPlayRank']),
                "Ranked Play Count {} Rank : #{:,}".format(df_info["country"][0],df_info.iloc[0]['RankedPlayJPRank']),

                "Total FC Count : {:,}".format(df_info.iloc[0]['TotalFC']),
                "","",
                "Ranked FC Count : {:,}".format(df_info.iloc[0]['RankedFC'])
                ))



    # PP
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["pp"], name="PP", line=dict(color="#646EFA",width=2)),row=1,col=1)
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["rank"], name="Rank", line=dict(color="#646EFA",width=2,dash='dashdot')),row=1,col=2)
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["countryRank"], name="{}-Rank".format(df_info["country"][0]), line=dict(color="#646EFA",width=2,dash='dot')), row=1, col=3)

    # Acc
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["AveRankedAcc"], name="Acc", line=dict(color="#ff4500",width=2)),row=2,col=1)
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["AveRankedAccRank"], name="Acc-Rank", line=dict(color="#ff4500",width=2,dash='dashdot')),row=2,col=2)
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["AveRankedAccJPRank"], name="Acc-{}-Rank".format(df_info["country"][0]), line=dict(color="#ff4500",width=2,dash='dot')),row=2,col=3)

    # TotalScore FC779F 1dd0a2
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["TotalScore"], name="TotalScore", line=dict(color="#1dd0a2",width=2)),row=3,col=1)
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["TotalScoreRank"], name="TotalScore-Rank", line=dict(color="#1dd0a2",width=2,dash='dashdot')),row=3,col=2)
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["TotalScoreJPRank"], name="TotalScore-{}-Rank".format(df_info["country"][0]), line=dict(color="#1dd0a2",width=2,dash='dot')),row=3,col=3)

    # RankedScore ff97ff b375fa
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["RankedScore"], name="RankedScore", line=dict(color="#b375fa",width=2)),row=4,col=1)
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["RankedScoreRank"], name="RankedScore-Rank", line=dict(color="#b375fa",width=2,dash='dashdot')),row=4,col=2)
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["RankedScoreJPRank"], name="RankedScore-{}-Rank".format(df_info["country"][0]), line=dict(color="#b375fa",width=2,dash='dot')),row=4,col=3)

    # TotalPlay ffcc66 fecb52
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["TotalPlay"], name="TotalPlay", line=dict(color="#ffcc66",width=2)),row=5,col=1)
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["TotalPlayRank"], name="TotalPlay-Rank", line=dict(color="#ffcc66",width=2,dash='dashdot')),row=5,col=2)
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["TotalPlayJPRank"], name="TotalPlay-{}-Rank".format(df_info["country"][0]), line=dict(color="#ffcc66",width=2,dash='dot')),row=5,col=3)

    # RankedPlay fecb52 fffd88 19d3f3
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["RankedPlay"], name="RankedPlay", line=dict(color="#19d3f3",width=2)),row=6,col=1)
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["RankedPlayRank"], name="RankedPlay-Rank", line=dict(color="#19d3f3",width=2,dash='dashdot')),row=6,col=2)
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["RankedPlayJPRank"], name="RankedPlay-{}-Rank".format(df_info["country"][0]), line=dict(color="#19d3f3",width=2,dash='dot')),row=6,col=3)

    # FC 1dd0a2 19d3f3 FC779F b6e880
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["TotalFC"], name="TotalFC", line=dict(color="#FC779F",width=2)),row=7,col=1)
    fig.add_trace(go.Scatter(x=df_summary["ScoreDateJa"], y=df_summary["RankedFC"], name="RankedFC", line=dict(color="#b6e880",width=2)),row=8,col=1)


    fig.update_layout(barmode="relative", title_text="Player Summary - {} - {} - {}".format(span, tz_ja.strftime("%Y.%m.%d"), df_info["name"][0]),
                    width=2000, height=1000,
                        # xaxis_rangeslider=dict(
                        #     visible=True,
                        #     #bgcolor='black',  # 背景色
                        #     #bordercolor='violet',  # 枠線の色
                        #     #borderwidth=6,  # 枠線の太さ
                        #     #thickness=0.3,  # レンジスライダーの厚さ
                        # ),
                    hovermode="x unified")


    #RankのY軸反転
    if rank_yaxis_reversed_is_enable:
        fig.update_layout(
            yaxis2_autorange="reversed"
            ,yaxis3_autorange="reversed"
            ,yaxis5_autorange="reversed"
            ,yaxis6_autorange="reversed"
            ,yaxis8_autorange="reversed"
            ,yaxis9_autorange="reversed"
            ,yaxis11_autorange="reversed"
            ,yaxis12_autorange="reversed"
            ,yaxis14_autorange="reversed"
            ,yaxis15_autorange="reversed"
            ,yaxis17_autorange="reversed"
            ,yaxis18_autorange="reversed"
        )

    display(df_info[cols_info_sort].style.set_table_styles(styles_info).format(style_format, na_rep="-"))
    fig.show(config=config)

<IPython.core.display.Javascript object>

,Pic,name,country,pp,rank,countryRank,role,TotalScore,RankedScore,AveRankedAcc,TotalPlay,RankedPlay,TotalFC,RankedFC,ReplayWatched,ScoreDateTz
0,,hatopop,JP,"9,758.78pp","#2,279",#162,-,"4,017,954,293","3,403,385,314",92.67%,"5,304",4410,"2,428","2,213",47,2025-06-15 01:58:09-07:00


In [ ]:
#@title ## History Tile


#@markdown ---
#@markdown <h4>Enable</h4>

history_tile_is_enable = True #@param {type:"boolean"}

#@markdown ---
#@markdown <h4>Setting</h4>

#@markdown <b>Limit</b>
data_max_size = 95 #@param {type:"slider", min:0, max:5000, step:1}

#@markdown <b>Target Setting</b>
target_global_rank = 50 #@param {type:"slider", min:0, max:5000, step:1}
target_pp = 270 #@param {type:"slider", min:0, max:1000, step:10}
target_weight = 9 #@param {type:"slider", min:0, max:1000, step:1}

#@markdown <b>image setting</b>
width = 1000 #@param {type:"slider", min:200, max:4000, step:100}
cover_image_size = 200 #@param {type:"slider", min:50, max:1000, step:50}

#@markdown <b>font size setting</b>

song_font_size = 22 #@param {type:"slider", min:3, max:50, step:1}
star_font_size = 18 #@param {type:"slider", min:3, max:50, step:1}
diff_font_size = 18 #@param {type:"slider", min:3, max:50, step:1}
acc_font_size = 18 #@param {type:"slider", min:3, max:50, step:1}
fc_font_size = 18 #@param {type:"slider", min:3, max:50, step:1}
pp_font_size = 22 #@param {type:"slider", min:3, max:50, step:1}
weight_font_size = 8 #@param {type:"slider", min:3, max:50, step:1}
rank_font_size = 22 #@param {type:"slider", min:3, max:50, step:1}
date_font_size = 12 #@param {type:"slider", min:3, max:50, step:1}

#@markdown <b>Visible Setting</b>

song_visible = True #@param {type:"boolean"}
star_visible = True #@param {type:"boolean"}
diff_visible = True #@param {type:"boolean"}
acc_visible = True #@param {type:"boolean"}
fc_visible = True #@param {type:"boolean"}
pp_visible = True #@param {type:"boolean"}
weight_visible = True #@param {type:"boolean"}
rank_visible = True #@param {type:"boolean"}
date_visible = True #@param {type:"boolean"}

#@markdown ---
#@markdown <h4>Filters</h4>
filtered_level_min = 0 #@param {type:"slider", min:0, max:20, step:1}
filtered_level_max = 15 #@param {type:"slider", min:0, max:20, step:1}
filtered_pp_min = 0 #@param {type:"slider", min:0, max:1000, step:10}
filtered_pp_max = 600 #@param {type:"slider", min:0, max:1000, step:10}
filtered_acc_min = 0 #@param {type:"slider", min:0, max:100, step:1}
filtered_acc_max = 100 #@param {type:"slider", min:0, max:100, step:1}
filtered_latest_is_enable = True #@param {type:"boolean"}
filtered_ranked_is_enable = False #param {type:"boolean"}
filtered_fullcombo_is_enable = False #@param {type:"boolean"}

#@markdown ---
#@markdown <h4>Sort</h4>

default_sort_key = "PP" #@param ["PP", "DateJa", "Acc", "Stars"]
song_keyword_filter = "" #@param {type:"string"}

# function
def str_difficulty(val):
    if val == "Easy":
        return 'E'
    elif val == "Normal":
        return 'N'
    elif val == "Hard":
        return 'H'
    elif val == "Expert":
        return 'Ex'
    elif val == "Expert+":
        return 'Ex+'
    elif val == "ExpertPlus":
        return 'Ex+'
    return '-'

def str_fc(val):
    if val == "FC":
        return 'FC'
    else:
        return ''

# FC color
def color_fc(val):
    if val == "FC":
        color = "#23D160"
    else:
        color = "black"
    return 'color: %s' % color

# pp color
def color_pp(val):
    if val >= target_pp:
        color = "red"
    else:
        color = "black"
    return 'color: %s' % color

# rank color
def color_rank(val):
    if val <= target_global_rank:
        color = "red"
    else:
        color = "black"
    return 'color: %s' % color

def color_latest(x):
    if  x == 1:
        color = "red"
    else:
        color = "black"
    return 'color: %s' % color

# weight
def format_weight(val):
    if val <= target_weight:
        return "".format(val)
    else:
        return "({:.1f}pp)".format(val)

def format_star(val):
    if val > 0:
        return "{:.2f}★".format(val)
    else:
        return ""
    return "{:.2f}★".format(val)

def format_acc(val):
    if val <= 100:
        return "{:.2f}%".format(val)
    else:
        return ""
    return "{:.2f}%".format(val)

def format_pp(val):
    if val > 0:
        return "{:.1f}pp".format(val)
    else:
        return ""
    return "{:.1f}pp".format(val)

def func_unrank(x):
    if x == 0:
        return None
    else:
        return x

def func_tile_css(_width=1000,
             _cover_image_size=200,
             _star_font_size=22,
             _diff_font_size=18,
             _acc_font_size=18,
             _fc_font_size=18,
             _pp_font_size=22,
             _weight_font_size=8,
             _rank_font_size=22,
             _date_font_size=18,
             _player_font_size=22
             ):
    return """
        <style>
        .flex_box {
            width:""" + str(_width) + """px;
            display: flex; /* フレックスボックスにする */;
            flex-wrap: wrap;
            padding:0px;
            margin:0px;
        }
        .flex_item {
            display: inline-block;
            position: relative;
            padding:0px 5px 0px 0px;
            margin:0px;
        }
        .flex_str_top, .flex_str_bottom_left, .flex_str_bottom_right{
            position: absolute;
            font-weight: bold;
            white-space:nowrap;
            overflow:hidden;
            color: #000;
            width:""" + str(_cover_image_size-10) + """px;
            text-shadow:1px 1px 0 #FFF, -1px -1px 0 #FFF,
                    -1px 1px 0 #FFF, 1px -1px 0 #FFF,
                    0px 1px 0 #FFF,  0-1px 0 #FFF,
                    -1px 0 0 #FFF, 1px 0 0 #FFF;
        }

        .flex_str_top {
            left: 5px;
            top: 5px;
        }

        .flex_str_bottom_left {
            left: 5px;
            bottom: 5px;
        }

        .flex_str_bottom_right {
            left: 5px;
            bottom: 5px;
        }

        .flex_star, .flex_acc, .flex_accrank, .flex_fc ,.flex_pp, .flex_weight, .flex_rank {
            padding:0px;
            margin:0px;
        }

        .flex_star{
            font-size: """ + str(_star_font_size) + """px;
        }

        .flex_diff{
            font-size: """ + str(_diff_font_size) + """px;
        }

        .flex_acc{
            font-size: """ + str(_acc_font_size) + """px;
        }

        .flex_fc {
            font-size: """ + str(_fc_font_size) + """px;
        }

        .flex_pp{
            font-size: """ + str(_pp_font_size) + """px;
        }

        .flex_weight{
            font-size: """ + str(_weight_font_size) + """px;
        }

        .flex_rank {
            font-size: """ + str(_rank_font_size) + """px;
            text-align:right;
        }

        .flex_date {
            font-size: """ + str(_date_font_size) + """px;
            text-align:right;
        }
        </style>
        """

def func_flex(x):
    div_song = "<div class='flex_song'>{}</div>".format(x[1]) if song_visible else ""
    span_star = "<span class='flex_star'>{}</span>".format(format_star(x[2])) if star_visible else ""
    span_diff = "<span= class='flex_diff'>{}</span>".format(str_difficulty(x[6])) if diff_visible else ""
    div_star_diff = "<div style='{}'>{} {}</div>".format(color_difficulty(x[6]), span_star, span_diff)

    span_acc = "<span class='flex_acc' style='{}'>{}</span>".format(color_acc_rank(x[4]),format_acc(x[3])) if acc_visible else ""
    span_fc = "<span class='flex_fc' style='{}'>{}</span>".format(color_fc(x[5]),str_fc(x[5])) if fc_visible else ""

    div_acc_fc = "<div>{} {}</div>".format(span_acc, span_fc)

    span_pp = "<span class='flex_pp' style='{}'>{}</span>".format(color_pp(x[7]), format_pp(x[7])) if pp_visible else ""
    span_weight = "<span class='flex_weight' style='{}'>{}</span>".format(color_pp(x[8]), format_weight(x[8])) if weight_visible else ""
    div_pp_weight = "<div>{} {}</div>".format(span_pp, span_weight)

    if date_visible:
        if filtered_latest_is_enable:
            div_date = "<div class='flex_date' style='{}'>{}/{} {}:{:02d}</div>".format(color_latest(x[16]), x[12], x[13], x[14], x[15])
        else:
            div_date = "<div class='flex_date' style='{}'>'{}/{}/{}</div>".format(color_latest(x[16]), str(x[11])[2:],x[12], x[13])
    else:
        div_date = ""

    div_rank = "<div class='flex_rank' style='{}'>#{:,.0f}</div>".format(color_rank(x[9]), x[9]) if rank_visible else ""

    span_diff_unrank = "<span= class='flex_star' style='{}'>{}</span>".format(color_difficulty(x[6]), str_difficulty(x[6])) if diff_visible else ""
    div_diff_fc = "<div style='{}'>{} {}</div>".format(color_difficulty(x[6]), span_diff, span_fc)

    return """
            <div class='flex_item'>{}<br/>
                <div class='flex_str_top'>
                    {}
                </div>
                <div class='flex_str_bottom_left'>
                    {}
                    {}
                    {}
                </div>
                <div class='flex_str_bottom_right'>
                    {}
                    {}
                </div>

            </div>
            """.format(x[0], div_song, div_star_diff, div_acc_fc, div_pp_weight, div_date, div_rank)


df_history = df_scores[(1==1)
    & (df_scores["Stars"] >= filtered_level_min)
    & (df_scores["Stars"] <= filtered_level_max)
    & (df_scores["PP"] >= filtered_pp_min)
    & (df_scores["PP"] <= filtered_pp_max)
    & (df_scores["Acc"] >= filtered_acc_min)
    & (df_scores["Acc"] <= filtered_acc_max)
    | (df_scores['Ranked']!=1)
    ]

if filtered_latest_is_enable:
    df_history = df_history[(df_history["Latest"] == 1)]

if filtered_ranked_is_enable:
    df_history = df_history[(df_history['Ranked']==1)]

if filtered_fullcombo_is_enable:
    df_history = df_history[(df_history['FC']=="FC")]

if len(song_keyword_filter) > 0:
    df_history = df_history[df_history["Song"].str.contains(song_keyword_filter, case=False)]

history_rows_count = len(df_history)

if history_tile_is_enable:
    print("{:,} results retrieved. * Max data size:{:,}\n".format(history_rows_count, data_max_size))
else:
    print("有効フラグが無効です。")

if history_rows_count > 0 and history_tile_is_enable:

    df_history_table = df_history.copy()

    df_history_table['Song'] = "<b>" + df_history_table["SongName"] + " " + df_history_table["SongSub"] + " </b><br /> " + df_history_table["SongAuthor"] + "<br />[" + df_history_table["LevelAuthor"] + "]"

    df_history_table['Year'] = df_history_table.index.year
    df_history_table['Month'] = df_history_table.index.month
    df_history_table['Day'] = df_history_table.index.day
    df_history_table['Hour'] = df_history_table.index.hour
    df_history_table['Minute'] = df_history_table.index.minute

    df_history_table = df_history_table

    df_history_table['Stars'] = df_history_table['Stars'].apply(func_unrank)

    df_flex_table = df_history_table.copy()

    df_flex_table = df_flex_table.sort_values([default_sort_key], ascending=False)

    df_flex_table["FlexCover"] = '<img src="https://cdn.scoresaber.com/covers/' + df_flex_table["Hash"] + '.png" style="width:{}px;height:{}px"/>'.format(cover_image_size,cover_image_size)

    df_flex_table["FlexItems"] =df_flex_table[['FlexCover','Song','Stars','Acc','AccRank','FC','Difficulty','PP', 'PPWeight','Rank','Ranked','Year','Month','Day','Hour','Minute','Latest']].apply(func_flex, axis=1)

    flex_head = """
                <div id="imageDIV"
                    class='flex_box_wrapper'>
                    <div class='flex_box'>
                """
    flex_content = " ".join(df_flex_table.reset_index()['FlexItems'][:data_max_size].to_list())
    flex_end = "</div></div>"

    img_css = func_tile_css(width,
             cover_image_size,
             star_font_size,
             diff_font_size,
             acc_font_size,
             fc_font_size,
             pp_font_size,
             weight_font_size,
             rank_font_size,
             date_font_size
             )

    html_str = flex_head + flex_content + flex_end + img_css

else:
    html_str = ""

HTML(html_str)

<IPython.core.display.Javascript object>

0 results retrieved. * Max data size:95



In [ ]:
#@title ## Level Cleared Progress

#@markdown ---
#@markdown <h4>Enable</h4>
level_cleared_progress_is_enable = True #@param {type:"boolean"}

#@markdown ---
#@markdown <b>Setting</b>
full_combo_progress_is_enable = True #param {type:"boolean"}
show_legends = True  # @param {type:"boolean"}
show_legend = False  # param {type:"boolean"}
fixed_scale = True  #param {type:"boolean"}
#markdown <b>Setting</b>
count_y_axis_max_scale = 1000 #@param {type:"number"}
width=800 #param {type:"number"}
height=800 #param {type:"number"}

graph_type_2nd_column = "By Songs" #@param ["By Songs", "By Played*"]


# plotly ファイル名設定
plot_name = "LevelClearedProgress"
config = {
    'toImageButtonOptions': {
        'format': 'png',
        'filename': "{}_{}_{}".format(plot_name, df_info["name"][0], tz_ja.strftime("%Y%m%d_%H%M%S")),
        'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
    }
}

if level_cleared_progress_is_enable:

    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    rows_count = 3

    plot_titles = [
                "Cleared Count", f"Cleared Rate - {graph_type_2nd_column}",
                "Acc Ranked Count", f"Acc Ranked Rate - {graph_type_2nd_column}",
                "Full Combo Count", f"Full Combo Rate - {graph_type_2nd_column}",
                ]

    fig = make_subplots(
        rows=rows_count, cols=2,
        horizontal_spacing=0.045,
        vertical_spacing=0.075,
        subplot_titles=(plot_titles)
        )

    # ---------------------------
    # Cleared
    # ---------------------------
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["AlreadyCleared"], name="AlreadyCleared", marker_color="#636EFA"),row=1,col=1)
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["RecentCleared"], name="RecentCleared", marker_color="#ff4500"),row=1,col=1)
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["NF"], name="NoFail", marker_color="#a0a0a0"),row=1,col=1)
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["NotCleared"], name="NotCleared", marker_color="#dcdcdc"),row=1,col=1)

    y_center_1 = round(0.7 * count_y_axis_max_scale)
    y_positions_1 = [round(i * count_y_axis_max_scale) for i in [0.8, 0.7, 0.6]]

    # Clear annotation
    fig.add_annotation(x=10,y=y_center_1, xref="x", yref="y",
            text="          ",
            borderpad=30,bgcolor="#FFFFFF",
            font=dict(size=12,color="#00CC00"), align="center",showarrow=False, opacity=0.8,row=1,col=1
            )

    fig.add_annotation(
            x=10,y=y_positions_1[0], xref="x", yref="y",
            text="Songs:{:,.0f}".format(int(df_progress["Song"].sum())),
            font=dict(size=12,color="#333333"),
            align="center",showarrow=False, opacity=0.8
            ,row=1,col=1
            )

    fig.add_annotation(
            x=10,y=y_positions_1[1], xref="x", yref="y",
            text="Cleared:{:,.0f}".format(int(df_progress["Cleared"].sum())),
            font=dict(size=12,color="#636EFA"),
            align="center",showarrow=False, opacity=0.8
            ,row=1,col=1
            )

    fig.add_annotation(
            x=10,y=y_positions_1[2], xref="x", yref="y",
            text="(Recent:{:,.0f})".format(int(df_progress["RecentCleared"].sum())),
            font=dict(size=12,color="#ff4500"),
            align="center",showarrow=False, opacity=0.8
            ,row=1,col=1
            )

    # Cleared - 2nd col
    if graph_type_2nd_column == "By Songs":
    # elif graph_type_2nd_column == "By Played*":
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["AlreadyClearedRate"], name="AlreadyClearedRate", marker_color="#636EFA", showlegend=show_legend),row=1,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["RecentClearedRate"], name="RecentClearedRate", marker_color="#ff4500", showlegend=show_legend),row=1,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["NFRate"], name="NoFailRate", marker_color="#a0a0a0", showlegend=show_legend),row=1,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["NotClearedRate"], name="NotClearedRate",marker_color="#dcdcdc", showlegend=show_legend),row=1,col=2)
    elif graph_type_2nd_column == "By Played*":
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["AlreadyClearedRateByPlayed"], name="AlreadyClearedRateByPlayed", marker_color="#636EFA", showlegend=show_legend),row=1,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["RecentClearedRateByPlayed"], name="RecentClearedRateByPlayed", marker_color="#ff4500", showlegend=show_legend),row=1,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["NFRateByPlayed"], name="NoFailRateByPlayed", marker_color="#a0a0a0", showlegend=show_legend),row=1,col=2)

    # ---------------------------
    # Acc Rank
    # ---------------------------
    # 凡例空白用のダミートレース
    fig.add_trace(go.Scatter(x=[0], y=[0], mode='lines', showlegend=True, name='---------------', opacity=0, hoverinfo='none'),row=2,col=1)
    # データ
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["SSS"], name="SSS", marker_color="#00ffff"),row=2,col=1)
    if ss_plus_is_enable:
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress[ss_plus], name=ss_plus, marker_color="#636EFA"),row=2,col=1)
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["SS"], name="SS", marker_color="#ff4500"),row=2,col=1)
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["S"], name="S", marker_color="#ffaaff"),row=2,col=1)
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["A"], name="A", marker_color="#ffcc66"),row=2,col=1)
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["B"], name="B", marker_color="#fffd88"),row=2,col=1)
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["Other"], name="Other",marker_color="#dcdcdc"),row=2,col=1)

    # Acc Rank - 2nd col
    if graph_type_2nd_column == "By Songs":
        # elif graph_type_2nd_column == "By Played*":
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["SSS-Rate"], name="SSS-Rate", marker_color="#00ffff", showlegend=show_legend),row=2,col=2)
        if ss_plus_is_enable:
            fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress[ss_plus_rate], name=ss_plus_rate, marker_color="#636EFA", showlegend=show_legend),row=2,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["SS-Rate"], name="SS-Rate", marker_color="#ff4500", showlegend=show_legend),row=2,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["S-Rate"], name="S-Rate", marker_color="#ffaaff", showlegend=show_legend),row=2,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["A-Rate"], name="A-Rate", marker_color="#ffcc66", showlegend=show_legend),row=2,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["B-Rate"], name="B-Rate", marker_color="#fffd88", showlegend=show_legend),row=2,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["Other-Rate"], name="Other-Rate",marker_color="#dcdcdc", showlegend=show_legend),row=2,col=2)
    elif graph_type_2nd_column == "By Played*":
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["SSS-RateByPlayed"], name="SSS-RateByPlayed", marker_color="#00ffff", showlegend=show_legend),row=2,col=2)
        if ss_plus_is_enable:
            fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress[ss_plus_rate+"ByPlayed"], name=ss_plus_rate, marker_color="#636EFA", showlegend=show_legend),row=2,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["SS-RateByPlayed"], name="SS-RateByPlayed", marker_color="#ff4500", showlegend=show_legend),row=2,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["S-RateByPlayed"], name="S-RateByPlayed", marker_color="#ffaaff", showlegend=show_legend),row=2,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["A-RateByPlayed"], name="A-RateByPlayed", marker_color="#ffcc66", showlegend=show_legend),row=2,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["B-RateByPlayed"], name="B-RateByPlayed", marker_color="#fffd88", showlegend=show_legend),row=2,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["Other-RateByPlayed"], name="Other-RateByPlayed",marker_color="#a0a0a0", showlegend=show_legend),row=2,col=2)


    y_center_2 = round(0.6 * count_y_axis_max_scale)
    y_positions_2 = [round(i * count_y_axis_max_scale) for i in [0.85, 0.75, 0.65, 0.55, 0.45, 0.35]]

    # Acc annotation
    fig.add_annotation(x=10,y=y_center_2, xref="x", yref="y",
            text="  ",
            borderpad=45,bgcolor="#FFFFFF",
            font=dict(size=12,color="#00CC00"), align="center",showarrow=False, opacity=0.8,row=2,col=1
            )

    if ss_plus_is_enable:
        fig.add_annotation(x=10,y=y_positions_2[0], xref="x", yref="y",
                text="SSS:{:,.0f}".format(int(df_progress["SSS"].sum())),
                font=dict(size=12,color="#00DDDD"), align="center",showarrow=False, opacity=0.8, row=2,col=1
                )

        fig.add_annotation(x=10,y=y_positions_2[1], xref="x", yref="y",
                text="{}:{:,.0f}".format(ss_plus,int(df_progress[ss_plus].sum())),
                font=dict(size=12,color="#636EFA"),align="center",showarrow=False, opacity=0.8, row=2,col=1
                )
    else:
        fig.add_annotation(x=10,y=y_positions_2[1], xref="x", yref="y",
            text="SSS:{:,.0f}".format(int(df_progress["SSS"].sum())),
            font=dict(size=12,color="#00DDDD"), align="center",showarrow=False, opacity=0.8, row=2,col=1
            )

    fig.add_annotation(x=10,y=y_positions_2[2], xref="x", yref="y",
            text="SS:{:,.0f}".format(int(df_progress["SS"].sum())),
            font=dict(size=12,color="#ff4500"),align="center",showarrow=False, opacity=0.8,row=2,col=1
            )

    fig.add_annotation(x=10,y=y_positions_2[3], xref="x", yref="y",
            text="S:{:,.0f}".format(int(df_progress["S"].sum())),
            font=dict(size=12,color="#ffaaff"),align="center",showarrow=False, opacity=0.8,row=2,col=1
            )

    fig.add_annotation(x=10,y=y_positions_2[4], xref="x", yref="y",
            text="A:{:,.0f}".format(int(df_progress["A"].sum())),
            font=dict(size=12,color="#ffbb55"),align="center",showarrow=False, opacity=0.8,row=2,col=1
            )

    fig.add_annotation(x=10,y=y_positions_2[5], xref="x", yref="y",
            text="B:{:,.0f}".format(int(df_progress["B"].sum())),
            font=dict(size=12,color="#ffed66"),align="center",showarrow=False, opacity=0.8,row=2,col=1
            )

    # ---------------------------
    # Full Combo
    # ---------------------------
    # 凡例空白用のダミートレース
    fig.add_trace(go.Scatter(x=[0], y=[0], mode='lines', showlegend=True, name='---------------', opacity=0, hoverinfo='none'),row=3,col=1)
    # データ
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["AlreadyFC"], name="AlreadyFC", marker_color="#00eecc"),row=3,col=1)
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["RecentFC"], name="RecentFC", marker_color="#ff4500"),row=3,col=1)
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["NotFC"], name="NotFC", marker_color="#a0a0a0"),row=3,col=1)
    fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["NotCleared"], name="NotCleared", marker_color="#dcdcdc"),row=3,col=1)

    # Full Combo - 2nd col
    if graph_type_2nd_column == "By Songs":
        # elif graph_type_2nd_column == "By Played*":
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["AlreadyFCRate"], name="AlreadyFCRate", marker_color="#00eecc", showlegend=show_legend),row=3,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["RecentFCRate"], name="RecentFCRate", marker_color="#ff4500", showlegend=show_legend),row=3,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["NotFCRate"], name="NotFCRate", marker_color="#a0a0a0", showlegend=show_legend),row=3,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["NotClearedRate"], name="NotClearedRate",marker_color="#dcdcdc", showlegend=show_legend),row=3,col=2)

    elif graph_type_2nd_column == "By Played*":
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["AlreadyFCRateByPlayed"], name="AlreadyFCRateByPlayed", marker_color="#00eecc", showlegend=show_legend),row=3,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["RecentFCRateByPlayed"], name="RecentFCRateByPlayed", marker_color="#ff4500", showlegend=show_legend),row=3,col=2)
        fig.add_trace(go.Bar(x=df_progress["Level"], y=df_progress["NotFCRateByPlayed"], name="NotFCRateByPlayed", marker_color="#a0a0a0", showlegend=show_legend),row=3,col=2)

    y_center_3 = round(0.7 * count_y_axis_max_scale)
    y_positions_3 = [round(i * count_y_axis_max_scale) for i in [0.8, 0.7, 0.6]]

    fig.add_annotation(x=10,y=y_center_3, xref="x", yref="y",
            text="          ",
            borderpad=30,bgcolor="#FFFFFF",
            font=dict(size=12,color="#00CC00"), align="center",showarrow=False, opacity=0.8,row=3,col=1
            )

    fig.add_annotation(x=10,y=y_positions_3[0], xref="x", yref="y",
            text="Songs:{:,.0f}".format(int(df_progress["Song"].sum())),
            font=dict(size=12,color="#333333"), align="center",showarrow=False, opacity=0.8 ,row=3,col=1
            )

    fig.add_annotation(x=10,y=y_positions_3[1], xref="x", yref="y",
            text="FC:{:,.0f}".format(int(df_progress["FC"].sum())),
            #borderpad=30,bgcolor="#FFFFFF",
            font=dict(size=12,color="#00CC00"), align="center",showarrow=False, opacity=0.8,row=3,col=1
            )

    fig.add_annotation(x=10,y=y_positions_3[2], xref="x", yref="y",
            text="(Recent:{:,.0f})".format(int(df_progress["RecentFC"].sum())),
            font=dict(size=12,color="#ff4500"),align="center",showarrow=False, opacity=0.8, row=3,col=1
            )


    if fixed_scale:
        # Y-axis tickvals and ticktext
        y_tickvals = [0, 20, 40, 60, 80, 100]
        y_ticktext = ["0", "20", "40", "60", "80", "100"]

        # Count
        fig.update_yaxes(range=[0, count_y_axis_max_scale], tickvals=[0, 200, 400, 600, 800, 1000], row=1, col=1)
        fig.update_yaxes(range=[0, count_y_axis_max_scale], tickvals=[0, 200, 400, 600, 800, 1000], row=2, col=1)
        fig.update_yaxes(range=[0, count_y_axis_max_scale], tickvals=[0, 200, 400, 600, 800, 1000], row=3, col=1)

        # Rate
        fig.update_yaxes(range=[0, 100], tickvals=y_tickvals, ticktext=y_ticktext, row=1, col=2)
        fig.update_yaxes(range=[0, 100], tickvals=y_tickvals, ticktext=y_ticktext, row=2, col=2)
        fig.update_yaxes(range=[0, 100], tickvals=y_tickvals, ticktext=y_ticktext, row=3, col=2)

        # Rate
        fig.update_yaxes(range=[0, 100], tickvals=y_tickvals, row=1, col=3)
        fig.update_yaxes(range=[0, 100], tickvals=y_tickvals, row=2, col=3)
        fig.update_yaxes(range=[0, 100], tickvals=y_tickvals, row=3, col=3)



    fig.update_layout(barmode="relative", title_text="Ranked - Level Cleared Progress - {} - {}".format(tz_ja.strftime("%Y.%m.%d"), df_info["name"][0]),
                      showlegend=show_legends,
                      width=width, height=height)
    fig.show(config=config)


<IPython.core.display.Javascript object>

In [ ]:
#@title Level Cleared Detail
#@markdown Rank譜面の星別クリア進捗詳細(df_progress)

#@markdown ---
#@markdown <h4>Enable</h4>
level_cleared_detail_is_enable = True #@param {type:"boolean"}
#markdown <b>Mode</b>
view_filter = "Clear&FC&Acc&NotClear" #@param ["Clear&FC", "Acc","Clear&FC&Acc","Clear&FC&Acc&NotClear","RecentClear&FC","NotClear" ,"All"]

ranked_count = df_progress["Song"].sum()
cleared_count = int(df_progress["Cleared"].sum())
nofail_count = int(df_progress["NF"].sum())
play_count = cleared_count + nofail_count
fc_count = int(df_progress["FC"].sum())
recent_cleared_count = int(df_progress["RecentCleared"].sum())

# 表示
if view_filter == "Clear&FC":
    cols_view = ["Song", "Cleared", "FC"]
elif view_filter == "Acc":
    if ss_plus_is_enable:
        cols_view = ["Song", "SSS", ss_plus, "SS", "S", "A", "B"]
    else:
        cols_view = ["Song", "SSS", "SS", "S", "A", "B"]

elif view_filter == "Clear&FC&Acc":
    if ss_plus_is_enable:
        cols_view = ["Song", "Cleared", "FC", "SSS", ss_plus, "SS", "S", "A", "B"]
    else:
        cols_view = ["Song", "Cleared", "FC", "SSS", "SS", "S", "A", "B"]
elif view_filter == "Clear&FC&Acc&NotClear":
    if ss_plus_is_enable:
        cols_view = ["Song", "Cleared", "FC", "SSS", ss_plus, "SS", "S", "A", "B", "NF","NotCleared"]
    else:
        cols_view = ["Song", "Cleared", "FC", "SSS", "SS", "S", "A", "B", "NF","NotCleared"]
elif view_filter == "RecentClear&FC":
    cols_view = ["Song", "Cleared", "RecentCleared", "FC", "RecentFC"]
elif view_filter == "NotClear":
    cols_view = ["Song", "Cleared", "NF","NotCleared"]
else:
    cols_view = df_progress.columns.values.tolist()
    cols_view.remove('Level')

def append_sum_row(df):
    return df.append(df.sum(numeric_only=True), ignore_index=True)

def append_sum_row_label(df):
    df.loc['Total'] = df.sum(numeric_only=True)
    return df

_df_progress = df_progress.copy()
_df_progress = _df_progress.set_index('Level')

df_progress_total = append_sum_row_label(_df_progress)

if not level_cleared_detail_is_enable:
    df_progress_total = df_progress_total.head(0)
    print('表示が有効ではありません。')

    # <H1>[BeatLeader] MyBSTile - {} - {}</H1>

caption_styles = [dict(selector="caption",
                       props=[
                            # ("text-align", "center"),
                            ("text-align", "lett"),
                              ("font-size", "150%"),
                            #   ("font-weight", "bold"),
                            #   ("padding", "3px 0 3px 3px"),
                            #   ("border-left", "4px solid #D90E44"),
                              ("border-bottom", "1px solid"),
                            #   ("color", 'black'),
                            #   ("text-decoration", 'underline')
                              ]
                       )
                  ]


df_progress_total[cols_view].rename_axis(None, axis=0).style.highlight_null(null_color="lightgray" # ).format(style_format)
    ).format(style_format).set_table_attributes("style='display:inline'"
    ).set_caption("Ranked - Level Cleared Detail - {} - {}".format(tz_ja.strftime("%Y.%m.%d"), df_info["name"][0])
    ).set_table_styles(caption_styles)

<IPython.core.display.Javascript object>

,Song,Cleared,FC,SSS,SS+95,SS,S,A,B,NF,NotCleared
0,8,8,8,0,8,0,0,0,0,0,0
1,3,3,3,0,3,0,0,0,0,0,0
2,23,22,22,0,22,0,0,0,0,0,1
3,323,312,310,0,312,0,0,0,0,0,11
4,"1,408","1,335","1,257",0,"1,197",136,2,0,0,0,73
5,890,810,481,0,349,459,2,0,0,0,80
6,504,462,102,0,31,410,21,0,0,0,42
7,475,437,26,0,3,364,68,2,0,2,36
8,489,442,4,0,0,217,222,3,0,4,43
9,449,343,0,0,0,34,283,26,0,16,90


In [ ]:
#@title ScatterPlot Analysis

#@markdown ---
#@markdown <h4>Enable</h4>
scatterplot_is_enable = True #@param {type:"boolean"}

#@markdown ---

#@markdown <h4>Parameters</h4>

col_x = "Stars" #@param ["PP", "DateJa", "Acc", "Stars","Rank"]
col_y = "Acc" #@param ["PP", "DateJa", "Acc", "Stars","Rank"]
size = "Stars" #@param ["PP", "Acc", "AccRank", "Stars"]
color = "LevelStr" #@param ["PP", "DateJa", "Acc", "LevelStr", "AccRank", "Stars", "Days"]

#@markdown ---
#@markdown <h4>Filters</h4>
filtered_star_min = 0 #@param {type:"slider", min:0, max:20, step:1}
filtered_star_max = 15 #@param {type:"slider", min:0, max:20, step:1}
filtered_pp_min = 0 #@param {type:"slider", min:0, max:1000, step:10}
filtered_pp_max = 1000 #@param {type:"slider", min:0, max:1000, step:10}
filtered_acc_min = 17 #@param {type:"slider", min:0, max:100, step:1}
filtered_acc_max = 100 #@param {type:"slider", min:0, max:100, step:1}


# plotly ファイル名設定
plot_name = "ScatterPlot"
config = {
    'toImageButtonOptions': {
        'format': 'png',
        'filename': "{}_{}_{}".format(plot_name, df_info["name"][0], tz_ja.strftime("%Y%m%d_%H%M%S")),
        'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
    }
}

if scatterplot_is_enable:
    import plotly.io as pio

    def _func_latest(x):
        if  x <= latest:
            return "latest {}day".format(latest)
        else:
            return "old".format(latest)

    def func_null_exclude(x):
        if len(x[0]) == 0:
            return 0
        else:
            return x[1]

    df_scatter = df_scores[(df_scores['Ranked']==1)
                    & (df_scores["PP"] >= filtered_pp_min)
                    & (df_scores["PP"] <= filtered_pp_max)
                    & (df_scores["Acc"] >= filtered_acc_min)
                    & (df_scores["Acc"] <= filtered_acc_max)
                    & (df_scores["Stars"] >= filtered_star_min)
                    & (df_scores["Stars"] <= filtered_star_max)]

    fig2 = px.scatter(df_scatter
                    , x=col_x, y=col_y, color=color,
                    size=size, hover_data=["PP","Acc","AccRank","Score","Miss","Bad","Song"],
                    width=800, height=800)

    fig2.update_layout(title="Ranked - ScatterPlot Analysis - {} - {}".format(tz_ja.strftime("%Y.%m.%d"), df_info["name"][0]), title_x=0.5,
                xaxis_title = "x:{} (markersize:{}, markercolor:{})".format(col_x, size, color), yaxis_title="y:{}".format(col_y)
                )

    fig2.show(config=config)

<IPython.core.display.Javascript object>

In [ ]:
# -*- coding: utf-8 -*-
"""
English version — BoxPlot Analysis (dynamic bin edges)
----------------------------------------------------
This cell replaces the original hard‑coded 131‑edge construction with a fully
parameterised approach.  It mirrors the Japanese cell fix, so both notebooks
stay in sync.  Simply copy & paste over the old cell, or import this file into
Colab.
"""

# @title ## BoxPlot Analysis (EN)
# @markdown ---
# @markdown <h4>Enable</h4>
boxplot_is_enable = True  # @param {type:"boolean"}

# @markdown ---
# @markdown <h4>x axis setting</h4>
# left / right borders for the X axis (Star difficulty)
x_range_min = 0    # @param {type:"slider", min:0, max:20, step:1}
x_range_max = 15   # @param {type:"slider", min:0, max:20, step:1}
# bin width (★)
x_range_step = 1.0  # @param {type:"slider", min:0, max:20, step:0.1}
# visual padding around the axis
x_padding = 0.5    # @param {type:"number"}

# @markdown ---
# @markdown <h4>y axis setting</h4>

# @markdown <b>Acc</b>
y_acc_range_min = 70   # @param {type:"slider", min:0, max:100, step:5}
y_acc_range_max = 100  # @param {type:"slider", min:0, max:100, step:5}
y_acc_padding   = 0    # @param {type:"number"}

# @markdown <b>PP</b>
y_pp_range_min = 0     # @param {type:"slider", min:0, max:1000, step:10}
y_pp_range_max = 600   # @param {type:"slider", min:0, max:1000, step:10}
y_pp_padding   = 0     # @param {type:"number"}

# @markdown ---
# @markdown <h4>fig setting</h4>

# @markdown <b>Size</b>
fig_height = 1000  # @param {type:"number"}
fig_width  = 1000  # @param {type:"number"}

# @markdown <b>Mode</b>
view_mode = "Separate"  # @param ["Separate", "Secondary"]
box_mode  = "Simple"    # @param ["Detail", "Simple"]

# ---------------------------------------------------------------------------
# Plotly export file‑name configuration
# ---------------------------------------------------------------------------
plot_name = "BoxPlot"
config = {
    "toImageButtonOptions": {
        "format": "png",
        "filename": f"{plot_name}_{df_info['name'][0]}_{tz_ja.strftime('%Y%m%d_%H%M%S')}",
        "scale"  : 1,
    }
}

# ---------------------------------------------------------------------------
# MAIN
# ---------------------------------------------------------------------------
if boxplot_is_enable:
    import numpy as np
    import pandas as pd
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    # Choose box‑plot style
    _box_mode = {"Detail": "sd", "Simple": True}.get(box_mode, True)

    # Filter to Ranked scores only
    df_scores_box = df_scores[df_scores["Ranked"] == 1].copy()

    # ---------------------------------------------------------------------
    # Build dynamic bin edges for the "Level+" column
    # ---------------------------------------------------------------------
    star_min, star_max, step = x_range_min, x_range_max, x_range_step

    # Safety guard — ensure step > 0 to avoid infinite loop
    if step <= 0:
        raise ValueError("x_range_step must be greater than 0")

    # Generate edges; +step ensures the upper bound is included
    bin_edges  = np.arange(star_min, star_max + step, step).round(3)
    bin_labels = bin_edges[:-1]

    # Assign bins — right=False => interval is [edge, next_edge)
    df_scores_box["Level+"] = pd.cut(
        df_scores_box["Stars"],
        bins   = bin_edges,
        labels = bin_labels,
        right  = False,
    )

    # ---------------------------------------------------------------------
    # Construct Figure
    # ---------------------------------------------------------------------
    if view_mode == "Secondary":
        fig = make_subplots(
            specs=[[{"secondary_y": True}]],
            horizontal_spacing=0.065,
            vertical_spacing=0.075,
        )

        # Acc vs PP in secondary axis mode
        fig.add_trace(
            go.Box(
                x=df_scores_box["Level+"],
                y=df_scores_box["Acc"],
                name="Acc",
                boxmean=_box_mode,
            ),
            secondary_y=False,
        )
        fig.add_trace(
            go.Box(
                x=df_scores_box["Level+"],
                y=df_scores_box["PP"],
                name="PP",
                boxmean=_box_mode,
            ),
            secondary_y=True,
        )

        fig.update_layout(
            boxmode="group",
            title_text=(
                f"Ranked - Level vs Acc/PP BoxPlot Analysis - {tz_ja.strftime('%Y.%m.%d')} - "
                f"{df_info['name'][0]}"
            ),
            xaxis_title=f"Level (step:{x_range_step})",
            yaxis_title="Acc",
            yaxis2_title="PP",
            width=fig_width,
            height=fig_height,
        )

        # Axis ranges
        fig.update_layout(
            xaxis={"range": (x_range_min - x_padding, x_range_max + x_padding)},
            yaxis={"range": (y_acc_range_min - y_acc_padding, y_acc_range_max + y_acc_padding)},
            yaxis2={"range": (y_pp_range_min - y_pp_padding, y_pp_range_max + y_pp_padding)},
        )

    else:  # Separate mode (default)
        fig = make_subplots(
            rows=2,
            cols=1,
            horizontal_spacing=0.065,
            vertical_spacing=0.075,
        )

        # (1) Accuracy boxes (all vs recent)
        fig.add_trace(
            go.Box(
                x=df_scores_box["Level+"],
                y=df_scores_box["Acc"],
                name="Acc",
                boxmean=_box_mode,
            ),
            row=1,
            col=1,
        )
        fig.add_trace(
            go.Box(
                x=df_scores_box[df_scores_box["Latest"] == 1]["Level+"],
                y=df_scores_box[df_scores_box["Latest"] == 1]["Acc"],
                name="RecentAcc",
                boxmean=_box_mode,
            ),
            row=1,
            col=1,
        )

        # (2) PP boxes (all vs recent)
        fig.add_trace(
            go.Box(
                x=df_scores_box["Level+"],
                y=df_scores_box["PP"],
                name="PP",
                boxmean=_box_mode,
            ),
            row=2,
            col=1,
        )
        fig.add_trace(
            go.Box(
                x=df_scores_box[df_scores_box["Latest"] == 1]["Level+"],
                y=df_scores_box[df_scores_box["Latest"] == 1]["PP"],
                name="RecentPP",
                boxmean=_box_mode,
            ),
            row=2,
            col=1,
        )

        # Layout
        fig.update_layout(
            boxmode="group",
            title_text=(
                f"Ranked - Level vs Acc/PP BoxPlot Analysis - {tz_ja.strftime('%Y.%m.%d')} - "
                f"{df_info['name'][0]}"
            ),
            xaxis_title=f"Level (step:{x_range_step})",
            xaxis2_title=f"Level (step:{x_range_step})",
            yaxis_title="Acc",
            yaxis2_title="PP",
            width=fig_width,
            height=fig_height,
        )

        # Axis ranges
        fig.update_layout(
            xaxis={"range": (x_range_min - x_padding, x_range_max + x_padding)},
            xaxis2={"range": (x_range_min - x_padding, x_range_max + x_padding)},
            yaxis={"range": (y_acc_range_min - y_acc_padding, y_acc_range_max + y_acc_padding)},
            yaxis2={"range": (y_pp_range_min - y_pp_padding, y_pp_range_max + y_pp_padding)},
        )

    # ---------------------------------------------------------------------
    # Show the figure
    # ---------------------------------------------------------------------
    fig.show(config=config)


<IPython.core.display.Javascript object>

In [ ]:
#@title Worst/Top N Playlist Download
#@markdown ---
#@markdown <h4>Enable</h4>
download_playlist_is_enable = False #@param {type:"boolean"}
#worst_scores_is_enable = True #@param {type:"boolean"}
worst_or_top = "Worst" #@param ["Worst", "Top"]
worst_target = "Acc" #@param ["Acc", "PP", "Rank"]
N =  3 #@param {type:"number"}

#@markdown ---
#@markdown <h4>Filters</h4>
filtered_level_min = 0 #@param {type:"slider", min:0, max:20, step:1}
filtered_level_max = 15 #@param {type:"slider", min:0, max:20, step:1}
filtered_pp_min = 0 #@param {type:"slider", min:0, max:1000, step:10}
filtered_pp_max = 1000 #@param {type:"slider", min:0, max:1000, step:10}
filtered_acc_min = 0 #@param {type:"slider", min:0, max:100, step:1}
filtered_acc_max = 100 #@param {type:"slider", min:0, max:100, step:1}
filtered_miss_min = 0 #@param {type:"number"}
filtered_miss_max = 5000 #@param {type:"number"}
not_fullcombo_filter_is_enable = False #@param {type:"boolean"}

if worst_or_top == "Top":
    asc=True
else:
    asc=False

if worst_target == "Rank":
    asc != asc

print(asc)

df_worst = df_scores.head(0)

if download_playlist_is_enable:
    df_filtered = df_scores[(1==1)
        & (df_scores["Stars"] >= filtered_level_min)
        & (df_scores["Stars"] <= filtered_level_max)
        & (df_scores["PP"] >= filtered_pp_min)
        & (df_scores["PP"] <= filtered_pp_max)
        & (df_scores["Acc"] >= filtered_acc_min)
        & (df_scores["Acc"] <= filtered_acc_max)
        & (df_scores["Miss"] + df_scores["Bad"] >= filtered_miss_min)
        & (df_scores["Miss"] + df_scores["Bad"] <= filtered_miss_max)
    ]

    if not_fullcombo_filter_is_enable:
        df_filtered = df_filtered[df_filtered['FC'] != "FC"]

    for level in range(filtered_level_min, filtered_level_max):
        df_worst = df_worst.append(df_filtered[df_filtered["Level"]==level].sort_values(worst_target, ascending=asc).head(N))#, ignore_index=True)


    if download_playlist_is_enable:

        # playlistの出力
        songs = []
        for i, x in df_worst.iterrows():
            songs += [{
                "songName": x["SongName"],
                "levelAuthorName": x["LevelAuthor"],
                "hash": x["Hash"],
                "levelid": f"custom_level_{x['Hash']}",
                "difficulties": [
                    {
                        "characteristic": "Standard",
                        "name": x["Difficulty"]
                    }
                ]
            }]

        playlist = {
            "playlistTitle": "{}_{}Scores".format(tz_ja.strftime("%Y.%m.%d"), worst_or_top),
            "playlistAuthor": "hatopop",
            "songs": songs
            #,"image": img
        }

        song_playlist_path = r"{}/{}_playlist_{}.json".format(data_path, worst_or_top, datetime.now().strftime("%Y%m%d"))

        with open(song_playlist_path, "w") as f:
            json.dump(playlist, f)

            files.download(song_playlist_path)

        print("Level {}-{}, {}{} Playlistを出力しました. 曲数:{}.".format(filtered_level_min, filtered_level_max, worst_or_top, N, df_worst["Play"].count()))

    df_worst["WorstTop"] = worst_or_top

df_worst[cols_score].sort_values(["Level","Acc"]).style.set_table_styles(styles_data).applymap(
            color_difficulty, subset=["Difficulty"]
        ).applymap(
            color_fc, subset=["FC"]
        ).applymap(
            color_acc_rank, subset=["AccRank"]
        ).format(style_worst_top
        , na_rep="-")



def func_unrank(x):
    if x == 0:
        return None
    else:
        return x

def func_song_table(x):
    return "<b>" + x.replace("/","</b><br />").replace("[","<br />[")

def func_diff(x):
    if x == "ExpertPlus":
        return "Expert+"
    else:
        return x

def font_bold(x):
    return "font-weight: bold;"

# df_history_table['Stars'] = df_history_table['Stars'].apply(func_unrank)

# df_history_table['Song'] = df_history_table['Song'].apply(func_song_table)

#df_history_table['Song'] = "<b>" + df_history_table["SongName"] + " " + df_history_table["SongSub"] + " </b><br /> " + df_history_table["SongAuthor"] + "<br />[" + df_history_table["LevelAuthor"] + "]"

# df_flex_table = df_history_table.copy()

# df_history_table = df_history_table[cols_latest_history].rename(columns={'Difficulty': 'Diff'})

# df_history_table['Diff'] = df_history_table['Diff'].apply(func_diff)

# df_worst[cols_worst_n].sort_values(["Level","Acc"]).style.set_table_styles(styles_data).applymap(
#             color_difficulty, subset=["Difficulty"]
#         ).applymap(
#             color_fc, subset=["FC"]
#         ).applymap(
#             font_bold, subset=["Stars","Acc","Rank","PP"]
#         ).applymap(
#             color_acc_rank, subset=["AccRank"]
#         ).format(style_worst_top
#         , na_rep="-")


#df_history_table['Stars'] = df_history_table['Stars'].apply(func_unrank)

# df_history_table['Song'] = df_history_table['Song'].apply(func_song_table)

#df_history_table['Song'] = "<b>" + df_history_table["SongName"] + " " + df_history_table["SongSub"] + " </b><br /> " + df_history_table["SongAuthor"] + "<br />[" + df_history_table["LevelAuthor"] + "]"

df_history_table = df_worst.copy()

df_history_table['Stars'] = df_history_table['Stars'].apply(func_unrank)

df_history_table['Song'] = "<b>" + df_history_table["SongName"] + " " + df_history_table["SongSub"] + " </b><br /> " + df_history_table["SongAuthor"] + "<br />[" + df_history_table["LevelAuthor"] + "]"

df_history_table = df_history_table[cols_worst_n]

df_history_table = df_history_table[cols_worst_n].rename(columns={'Difficulty': 'Diff'})

df_history_table['Diff'] = df_history_table['Diff'].apply(func_diff)

df_history_table.sort_values(["Level","Acc"]).style.set_table_styles(styles_data).applymap(
            color_difficulty, subset=["Diff"]
        ).applymap(
            color_fc, subset=["FC"]
        ).applymap(
            font_bold, subset=["Stars","Acc","Rank","PP"]
        ).applymap(
            color_acc_rank, subset=["AccRank"]
        ).format(style_worst_top
        , na_rep="-")


<IPython.core.display.Javascript object>

False


,Cover,Song,Diff,Level,Stars,Acc,AccRank,FC,Rank,PP,Miss,Bad,Combo,Score
DateJa,,,,,,,,,,,,,,
